# 面向目标架构案例实践(下)

## 1. 自动代码审查流程

In [47]:
import os
import openai
from openai import OpenAI
import shutil

import json
import io
import inspect
import requests
import re
import random
import string

import openai
from openai import OpenAI
import os

## 初始化客户端
api_key = os.getenv("ZHIPU_API_KEY")

from zhipuai import ZhipuAI
client = ZhipuAI(api_key=api_key)

### 1.1 大模型debug能力测试

In [3]:
def print_multi():
    for i in range(1, 10):  # 控制行数，从1到9
        for j in range(1, i+1):  # 控制每行的列数，从1到i
            print(f"{j}x{i}={i*j}", end="\t")  # 打印乘法表达式，使用\t进行分隔
        print()  # 每完成一行的输出后换行

In [5]:
print_multi()

1x1=1	
1x2=2	2x2=4	
1x3=3	2x3=6	3x3=9	
1x4=4	2x4=8	3x4=12	4x4=16	
1x5=5	2x5=10	3x5=15	4x5=20	5x5=25	
1x6=6	2x6=12	3x6=18	4x6=24	5x6=30	6x6=36	
1x7=7	2x7=14	3x7=21	4x7=28	5x7=35	6x7=42	7x7=49	
1x8=8	2x8=16	3x8=24	4x8=32	5x8=40	6x8=48	7x8=56	8x8=64	
1x9=9	2x9=18	3x9=27	4x9=36	5x9=45	6x9=54	7x9=63	8x9=72	9x9=81	


In [6]:
content="""
def print_multi():
    for i in range(1, 10):  # 控制行数，从1到9
        for j in range(1, i+1):  # 控制每行的列数，从1到i
            print(f"{j}x{i}={i j}", end="\t")  # 打印乘法表达式，使用\t进行分隔
        print()  # 每完成一行的输出后换行
"""

In [7]:
messages = [{"role": "user",  "content": "现有函数如下：%s，该函数是否有和问题，应该如何修改？" % content}]

In [10]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=messages)

response_message = response.choices[0].message

In [28]:
def extract_code(s):
    """
    如果输入的字符串s是一个包含Python代码的Markdown格式字符串，提取出代码部分。
    否则，返回原字符串。

    参数:
    s: 输入的字符串。

    返回:
    提取出的代码部分，或原字符串。
    """
    # 判断字符串是否是Markdown格式
    if '```python' in s or 'Python' in s or'PYTHON' in s:
        # 找到代码块的开始和结束位置
        code_start = s.find('def')
        code_end = s.find('```\n', code_start)
        # 提取代码部分
        code = s[code_start:code_end]
    else:
        # 如果字符串不是Markdown格式，返回原字符串
        code = s

    return code

In [15]:
python_code=extract_code(response_message.content)

In [16]:
print(python_code)

def print_multi():
    for i in range(1, 10):  # 控制行数，从1到9
        for j in range(1, i+1):  # 控制每行的列数，从1到i
            # 使用空格分隔，并且修正了格式化字符串的错误
            print(f"{j}x{i}={i*j}", end=" ")
        print()  # 每完成一行的输出后换行



In [17]:
exec(python_code)

In [19]:
print_multi()

1x1=1 
1x2=2 2x2=4 
1x3=3 2x3=6 3x3=9 
1x4=4 2x4=8 3x4=12 4x4=16 
1x5=5 2x5=10 3x5=15 4x5=20 5x5=25 
1x6=6 2x6=12 3x6=18 4x6=24 5x6=30 6x6=36 
1x7=7 2x7=14 3x7=21 4x7=28 5x7=35 6x7=42 7x7=49 
1x8=8 2x8=16 3x8=24 4x8=32 5x8=40 6x8=48 7x8=56 8x8=64 
1x9=9 2x9=18 3x9=27 4x9=36 5x9=45 6x9=54 7x9=63 8x9=72 9x9=81 


总结：我们看到大模型的确有代码修复的能力，但是我们看到的是纯算法的代码，我们继续测试带有业务性质的代码逻辑。

这个地方，我们准备了一个错误的函数，我人为改错的。里面的userId,在函数里面传递传错了，我们看一下大模型是否能正常修改。

In [20]:
with open('./error_function_test/get_latest_email/get_latest_email_module.py' , encoding='utf-8') as f:
    content = f.read()

In [21]:
print(content)

def get_latest_email(userId):
    """
    获取最新的一封邮件
    参数:
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    """
    mysql_pw="gpt"
    
    connection = pymysql.connect(
    host='localhost',  # 数据库地址
    user='gpt',  # 数据库用户名
    passwd=mysql_pw,  # 数据库密码
    db='my_mail',  # 数据库名
    charset='utf8'  # 字符集选择utf8   
    )
    sql_query=f"SELECT * FROM mailbox WHERE userid = '{me}' ORDER BY date DESC LIMIT 1";

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        cursor.close()
    
    column_names = [desc[0] for desc in cursor.description]

    # 使用results和column_names创建DataFrame
    df = pd.DataFrame(results, columns=column_names)
        
    return df.to_json(orient = "records")  



In [23]:
messages = [{"role": "user",  "content": "现有函数如下：%s，该函数是否有和问题，应该如何修改？" % content}]

In [24]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=messages)

response_message = response.choices[0].message

In [26]:
print(response_message.content)

该函数存在一些潜在问题和可以改进的地方：

1. `userId` 参数未在函数体内正确使用，而是使用了硬编码的 `'me'`。
2. 直接使用 `f-string` 将变量 `userId` 插入到 SQL 查询中是不安全的，这可能导致 SQL 注入风险。
3. `pymysql` 应该在 `try` 块外创建连接，并在 `finally` 块中关闭连接。
4. 函数的注释中提到了返回一个列表，但实际返回的是 JSON 字符串。
5. `mysql_pw` 应该从环境变量或配置文件中读取，而不是硬编码在函数中。
6. 函数名和变量名应遵循 Python 的命名约定（如使用 `snake_case`）。

以下是修改后的代码：

```python
import pymysql
import pandas as pd
import json

def get_latest_email(user_id: str):
    """
    获取最新的一封邮件
    
    参数:
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。
    
    返回:
    一个字典，代表一封邮件。字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    如果没有找到邮件，返回空字典。
    """
    # 从环境变量或配置文件中读取数据库配置
    host = 'localhost'
    user = 'gpt'
    password = "your_database_password"  # 不要在这里硬编码密码
    db = 'my_mail'
    charset = 'utf8'
    
    # 创建连接应该在try块外部
    connection = pymysql.connect(host=host, user=user, passwd=password, db=db, charset=charset)
    
    # 使用参数化查询避免SQL注入
    sql_query = "SELECT * FROM mai

In [2]:
import pymysql
import pandas as pd
import json

def get_latest_email(user_id: str):
    """
    获取最新的一封邮件
    
    参数:
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。
    
    返回:
    一个字典，代表一封邮件。字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    如果没有找到邮件，返回空字典。
    """
    # 从环境变量或配置文件中读取数据库配置
    host = 'localhost'
    user = 'gpt'
    password = "gpt"  # 不要在这里硬编码密码
    db = 'my_mail'
    charset = 'utf8'
    
    # 创建连接应该在try块外部
    connection = pymysql.connect(host=host, user=user, passwd=password, db=db, charset=charset)
    
    # 使用参数化查询避免SQL注入
    sql_query = "SELECT * FROM mailbox WHERE userid = %s ORDER BY date DESC LIMIT 1"
    
    try:
        with connection.cursor() as cursor:
            cursor.execute(sql_query, (user_id,))
            result = cursor.fetchone()
            if result:
                # 使用列名和查询结果生成字典
                column_names = [desc[0] for desc in cursor.description]
                email_data = dict(zip(column_names, result))
            else:
                email_data = {}
    finally:
        # 确保关闭连接
        connection.close()
    
    # 返回字典而不是JSON字符串
    return email_data

In [3]:
latest_email = get_latest_email('me')
print(latest_email)

{'userid': 'me', 'from_userid': '西门吹雪', 'date': '2024-06-07', 'subject': '开会', 'snippet': '明天10点到海淀区叶青大厦5楼502开会', 'status': '已读', 'label': '开会'}


总结：看到大模型对这种有业务性质的代码，也照样能进行改正，我们可以相信大模型。

### 1.2 审查模型构建方案剖析

&emsp;&emsp;尽管上述示例中，我们只提供了函数代码信息，模型即可顺利找出代码问题并进行了有效修改，但实际上，由于此时用于审查的大模型（以下简称审查模型）并不知道该函数创建的前后信息，有时修改之后的模型可能也无法达到要求。例如审查模型并不知道我们业务的背景，我们目前的业务只是我们自己模拟的一个业务而已。

同时我们需要函数的输入和输出都是字符串格式等信息，因此审查模型在修改时也很有可能修改得到功能正常但不满足这些背景要求的函数。此外由于在进行函数创建时，为了积累成功函数的提示示例，我们不仅需要功能正常的函数代码，同时也需要各阶段的提示内容，因此对于一个报错函数，仅仅修改代码可能并不足够。基于此，我们不仅需要让审查模型审查代码，同时也需要让其审查整个提示流程，并在必要的时候对提示词进行修改。以此才能真正达到审查之目的，最终修改的结果才能直接使用。

- 我们怎么样让大模型帮我们全流程审核代码呢？

**预训练，微调**：这些方案需要的数据量很大，我们压根也没有那么多的样本的数据，而且这种方案门槛很高，成本也很高。所以不现实。

**One-Shot，Few-Shot**：这些方案在简单场景下有用，复杂场景就不管用了，而且如果我们Few-shot的量大的话，我们的上文的Token就不够用，所以这个方案也行不通。

**外挂本地知识库**：大模型本身能力就很强，而且懂的也很多，所以我们换个思路，我们搞一个本地知识库，在本地知识库里激发这个大模型的能力，从而引导这个大模型完成审查任务。

- 检查大模型本身具备的知识

&emsp;&emsp;如何引导和启发一个大语言模型去理解一些陌生的事物？就像老师教学一样，有两个简单高效的方法，其一是是用其已知的知识去类比未知的知识，其二则是将陌生事物视为某些一般概念的特例。例如，当前的外部挂载文档非常重要的一个作用就是要让大语言模型理解现在要审查的多段提示，即要引导大语言模型理解什么是多段提示，但如果从头开始进行相关概念介绍（例如从头介绍两个阶段的提示分别是什么、什么是LtM、什么是问题拆解等）会非常的繁琐且效率很低，因此这里我们可以将多段提示比喻成推理链，借助大语言模型本身对推理链的理解来理解什么是多段提示。当然，这里我们需要简单测试下大语言模型本身是否知道什么是推理链：

In [6]:
messages = [{"role": "user",  "content": "请问什么是推理链?"}]

In [7]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=messages)

response.choices[0].message.content

'推理链（Chain of Thought，简称CoT）是一种在大语言模型中用于提升复杂推理任务性能的方法。它主要涉及在生成最终答案之前，让模型明确地输出推理过程中的中间步骤。这种策略的核心思想是模拟人类解决问题时的思考过程，即通过一系列的逻辑推理步骤来逐步解析问题并得出结论。\n\n在使用推理链时，模型会被提供一些包含推理步骤的示例，这些示例展示了如何从问题的提出到逐步解决问题，并最终得到答案的过程。通过这样的示例，模型学习到不仅仅关注答案本身，而是要重视达到答案的推理路径。\n\n推理链有两种主要类型：\n\n1. **Few-Shot CoT**：在这种模式下，模型通过观察并学习少量的包含推理步骤的示例来学会推理。这些示例通常由人工标注提供，作为推理过程的示范。\n\n2. **Zero-Shot CoT**：与Few-Shot不同，Zero-Shot CoT不依赖于人工提供的示例。模型直接被指示或者通过内置的指令来生成一系列的推理步骤，而不需要先前的示范。\n\n推理链有助于提升模型在算术推理、常识推理和符号推理等领域的性能。它不仅能够提高解题的正确率，而且使模型的推理过程更加透明和可解释。\n\n然而，推理链的使用也面临一些挑战，比如需要平衡推理步骤的数量以及确保示例质量的问题。过多的推理步骤可能会导致模型混淆，而质量低下的示例则会影响模型的学习效果。\n\n综上所述，推理链是一种促进大型语言模型进行类似人类思维方式的推理策略，旨在通过逐步分析问题来提高模型解决复杂问题的能力。'

而在外部挂载文档中，我们就可以简单将多段提示看成是一种推理链，并且是一种非常特殊的推理链，其特殊之处在于这个推理链专门用于创建一系列关于邮箱的相关函数。这里我们进一步验证如果将多段提示视作推理链的话，大语言模型能否直接理解：

In [8]:
function_name = 'get_latest_email'

In [9]:
with open('./functions/tested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
    msg = json.load(f)

In [10]:
chain_of_prompt = "以下是个一个成功运行的推理链条，推理分为三个阶段，各阶段推理内容如下：%s。" % msg

In [12]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=[{"role": "user",  "content": "以下是一个推理链，用于创建一个满足某个功能的函数，推理链如下：%s。请帮我介绍下这个推理链" % chain_of_prompt}]
)

print(response.choices[0].message.content)

这个推理链是一个包含三个阶段的对话过程，目的是为了创建一个Python函数来查看指定邮箱中的最新一封邮件内容。下面是对每个阶段的介绍：

### 第一阶段(stage1_CD)
- **角色**：用户（User）和助手（Assistant）
- **内容**：
  - 用户请求助手帮忙查看名为`mailbox`的邮箱中的最新一封邮件。
  - 助手分析了用户的需求，并指出可以作为函数参数的信息是“查看谁的邮箱”。

### 第二阶段(stage1_CM)
- **角色**：用户（User）和助手（Assistant）
- **内容**：
  - 用户重申了之前的请求，并采纳了助手的分析结果，明确了函数参数的需求。
  - 助手根据用户的具体要求，提出要创建一个Python函数，并列出了函数的四个要求：
    1. 函数参数`userId`，默认为'me'表示查看自己的邮件。
    2. 函数返回结果必须是一个JSON格式的对象，包含最后一封邮件的信息。
    3. 所有功能需要封装在一个函数内。
    4. 函数内部需要包含中文编写的详细说明文档。

### 第三阶段(stage2)
- **角色**：用户（User）和助手（Assistant）
- **内容**：
  - 用户再次提出根据之前要求编写函数的请求。
  - 助手提供了一个名为`get_latest_email`的Python函数实现。这个函数使用pymysql连接数据库，查询名为`my_mail`的数据库中的`mailbox`表，并根据日期降序排列，取最新的一封邮件。函数还使用Pandas库来处理查询结果，并将结果转换为JSON格式返回。

### 注意事项：
1. 在这个推理链中，助手提供的代码示例假设邮箱数据存储在本地MySQL数据库的`my_mail`数据库的`mailbox`表中，这可能并不符合实际应用场景，实际情况下可能需要使用IMAP等协议来连接实际的邮箱服务。
2. 代码示例中的敏感信息（如数据库密码）被硬编码在函数中，这是不安全的做法，实际应用中应该使用环境变量或其他安全的方式来存储敏感信息。
3. 函数说明文档中提到的返回字段（例如`from_userid`、`date`等）并没有在代码的返回结果中明确体现，实际实现时应该保证文档和代码实现的一致性。

这个推理链展示了从用

赞，这个效果比我预想的还要好，看来大模型能理解我们的这个推理链了。

### 1.3 审查模型构建方案落地

In [13]:
with open('./data/推理链.md', 'r', encoding='utf-8') as f:
    md_content = f.read()
    
print(md_content)

1.什么是邮箱项目？邮件项目本身由一系列的程序组成，核心功能是根据用户自然语言描述进行相应的邮箱操作，如查阅邮件、收发邮件等。该场景是我们自己构建的一个场景。
主要实现的是对我们my_mail数据库里的数据进行增删查改。

2.什么是推理链？在邮箱项目中，推理链特指将用户的需求逐步推导为python函数的过程，也就是说，每个推理链的输入都是用户需求，而推理的结果则是一个可以满足用户需求的函数。每个推理链都有三个推理环节，且都以json格式表示，例如，以下是一个正确的推理链A：

```json
{"stage1_CD": [{"role": "user", "content": "\u8bf7\u5e2e\u6211\u67e5\u4e0bmailbox\u90ae\u7bb1\u91cc\u6700\u65b0\u4e00\u5c01\u90ae\u4ef6\u5185\u5bb9\u3002"}, {"role": "assistant", "content": "\u5f53\u524d\u9700\u6c42\u4e2d\u53ef\u4ee5\u4f5c\u4e3a\u51fd\u6570\u53c2\u6570\u7684\u662f\uff1a1.\u67e5\u770b\u8c01\u7684\u90ae\u7bb1\u3002"}], "stage1_CM": [{"role": "user", "content": "\u8bf7\u5e2e\u6211\u67e5\u4e0bmailbox\u90ae\u7bb1\u91cc\u6700\u65b0\u4e00\u5c01\u90ae\u4ef6\u5185\u5bb9\u3002\u5f53\u524d\u9700\u6c42\u4e2d\u53ef\u4ee5\u4f5c\u4e3a\u51fd\u6570\u53c2\u6570\u7684\u662f\uff1a1.\u67e5\u770b\u8c01\u7684\u90ae\u7bb1\u3002"}, {"role": "assistant", "content": "\u8bf7\u5e2e\u6211\u7f16\u5199\u4e00\u4e2apython\u51fd\u6570\uff0c\u7528\u4e8e\u67e5

&emsp;&emsp;这里我们可以简单总结外部挂载文档的5方面信息：1.说明当前项目背景（包括编程要求等）；2.说明审查对象，包括审查对象的性质和形式等；3.说明审查的目标；4.进行身份设置；5.说明审查之后的输出结果。主要内容不要太多，不然没办法给到system

### 1.4 审查模型效果验证

In [15]:
function_name = 'get_latest_email'

In [16]:
with open('./error_function_test/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
    msg = json.load(f)

In [18]:
wrong_code = msg['stage2'][1]['content']
print(wrong_code)

def get_latest_email(userId):
    """
    获取最新的一封邮件
    参数:
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    """
    mysql_pw="gpt"
    
    connection = pymysql.connect(
    host='localhost',  # 数据库地址
    user='gpt',  # 数据库用户名
    passwd=mysql_pw,  # 数据库密码
    db='my_mail',  # 数据库名
    charset='utf8'  # 字符集选择utf8   
    )
    sql_query=f"SELECT * FROM mailbox WHERE userid = '{me}' ORDER BY date DESC LIMIT 1";

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        cursor.close()
    
    column_names = [desc[0] for desc in cursor.description]

    # 使用results和column_names创建DataFrame
    df = pd.DataFrame(results, columns=column_names)
        
    return df.to_json(orient = "records")  



In [19]:
exec(wrong_code)

In [20]:
get_latest_email(userId='me')

NameError: name 'me' is not defined

目前代码是运行不了的

我们尝试用审查模型对齐修改

In [21]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=[
                            {"role": "system",  "content": md_content},
                            {"role": "user", "content": '请问什么是推理链？'}
                        ]
)


In [22]:

print(response.choices[0].message.content)

推理链是一种在人工智能和机器学习中使用的概念，特别是在自然语言处理和大型语言模型的应用中。它指的是一系列的逻辑步骤或推理步骤，这些步骤将原始问题或请求逐步转化为最终的解答或操作。推理链的目的是模拟人类解决问题时的思考过程，将复杂的任务分解为更小、更易于管理的部分，从而帮助模型更准确地理解和执行用户的意图。

在您提供的邮箱项目上下文中，推理链用于将用户的自然语言描述（例如，“请帮我查看邮箱里最新的一封邮件内容”）转化为一系列的编程操作，最终形成一个可以执行的Python函数。这样，用户的需求就可以通过模型生成的代码在技术上得到实现。

现在，如果您能提供一个错误的推理链示例，我将帮助您进行审查和修改。请您提供需要修改的推理链。


然后我们引导该模型对此前错误的提示流程进行审查和修改：

In [85]:
msg_str = json.dumps(msg)
print(msg_str)

{"stage1_CD": [{"role": "user", "content": "\u8bf7\u5e2e\u6211\u67e5\u4e0bmailbox\u90ae\u7bb1\u91cc\u6700\u65b0\u4e00\u5c01\u90ae\u4ef6\u5185\u5bb9\u3002"}, {"role": "assistant", "content": "\u5f53\u524d\u9700\u6c42\u4e2d\u53ef\u4ee5\u4f5c\u4e3a\u51fd\u6570\u53c2\u6570\u7684\u662f\uff1a1.\u67e5\u770b\u8c01\u7684\u90ae\u7bb1\u3002"}], "stage1_CM": [{"role": "user", "content": "\u8bf7\u5e2e\u6211\u67e5\u4e0bmailbox\u90ae\u7bb1\u91cc\u6700\u65b0\u4e00\u5c01\u90ae\u4ef6\u5185\u5bb9\u3002\u5f53\u524d\u9700\u6c42\u4e2d\u53ef\u4ee5\u4f5c\u4e3a\u51fd\u6570\u53c2\u6570\u7684\u662f\uff1a1.\u67e5\u770b\u8c01\u7684\u90ae\u7bb1\u3002"}, {"role": "assistant", "content": "\u8bf7\u5e2e\u6211\u7f16\u5199\u4e00\u4e2apython\u51fd\u6570\uff0c\u7528\u4e8e\u67e5\u770b\u6211\u7684mailbox\u90ae\u7bb1\u4e2d\u6700\u540e\u4e00\u5c01\u90ae\u4ef6\u4fe1\u606f\uff0c\u51fd\u6570\u8981\u6c42\u5982\u4e0b\uff1a                 1.\u51fd\u6570\u53c2\u6570userId\uff0cuserId\u662f\u5b57\u7b26\u4e32\u53c2\u6570\uff0c\u9ed8\u

In [24]:
'以下是一个错误的邮箱项目的推理链，请你按照要求对其进行修改：%s' % msg_str

'以下是一个错误的邮箱项目的推理链，请你按照要求对其进行修改：{"stage1_CD": [{"role": "user", "content": "\\u8bf7\\u5e2e\\u6211\\u67e5\\u4e0bmailbox\\u90ae\\u7bb1\\u91cc\\u6700\\u65b0\\u4e00\\u5c01\\u90ae\\u4ef6\\u5185\\u5bb9\\u3002"}, {"role": "assistant", "content": "\\u5f53\\u524d\\u9700\\u6c42\\u4e2d\\u53ef\\u4ee5\\u4f5c\\u4e3a\\u51fd\\u6570\\u53c2\\u6570\\u7684\\u662f\\uff1a1.\\u67e5\\u770b\\u8c01\\u7684\\u90ae\\u7bb1\\u3002"}], "stage1_CM": [{"role": "user", "content": "\\u8bf7\\u5e2e\\u6211\\u67e5\\u4e0bmailbox\\u90ae\\u7bb1\\u91cc\\u6700\\u65b0\\u4e00\\u5c01\\u90ae\\u4ef6\\u5185\\u5bb9\\u3002\\u5f53\\u524d\\u9700\\u6c42\\u4e2d\\u53ef\\u4ee5\\u4f5c\\u4e3a\\u51fd\\u6570\\u53c2\\u6570\\u7684\\u662f\\uff1a1.\\u67e5\\u770b\\u8c01\\u7684\\u90ae\\u7bb1\\u3002"}, {"role": "assistant", "content": "\\u8bf7\\u5e2e\\u6211\\u7f16\\u5199\\u4e00\\u4e2apython\\u51fd\\u6570\\uff0c\\u7528\\u4e8e\\u67e5\\u770b\\u6211\\u7684mailbox\\u90ae\\u7bb1\\u4e2d\\u6700\\u540e\\u4e00\\u5c01\\u90ae\\u4ef6\\u4fe1\\u606f\\uff0c\\u51fd\\u6570

In [96]:
with open('./data/my_mail数据字典.md', 'r', encoding='utf-8') as f:
    data_content = f.read()

In [122]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=[
                            {"role": "system", "content": data_content},
                            {"role": "system", "content": md_content},
                            {"role": "user", "content": '以下是一个错误的邮箱项目的推理链，请你按照要求对其进行修改：%s' % msg_str}
                          ]
)


In [123]:
print(response.choices[0].message.content)

错误之处在于:

- 在最后一步中，assistant提供了一个Python函数，这个函数中有一个错误，即在sql查询中，userid应等于函数参数的userid，但是现在写的是me。对于这个错误的修改，我们应该将sql_query中的me修改为userId。

修改后的推理链应如下：

```json
{
"stage1_CD": [
    {"role": "user", "content": "请帮我查下mailbox邮件箱里最新一封邮件内容。"},
    {"role": "assistant", "content": "当前需求中可以作为函数参数的是：1.查看谁的邮件。"}
], 
"stage1_CM": [
    {"role": "user", "content": "请帮我查下mailbox邮件箱里最新一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮件。"}, 
    {"role": "assistant", "content": "请帮我编写一个python函数，用于查看我在mailbox邮件箱中最新一封邮件的信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数里；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"}], 
"stage2": [
    {"role": "user", "content": "请帮我编写一个python函数，用于查看我在mailbox邮件箱中最新一封邮件的信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数里；      

In [141]:
import re

def extract_json(s):
    pattern = r'```[jJ][sS][oO][nN]\s*({.*?})\s*```'
    match = re.search(pattern, s, re.DOTALL)
    if match:
        return match.group(1)
    else:
        return s

In [142]:
modified_result=response.choices[0].message.content

In [143]:
json_str=extract_json(modified_result)
print(json_str)

{
"stage1_CD": [
    {"role": "user", "content": "请帮我查下mailbox邮件箱里最新一封邮件内容。"},
    {"role": "assistant", "content": "当前需求中可以作为函数参数的是：1.查看谁的邮件。"}
], 
"stage1_CM": [
    {"role": "user", "content": "请帮我查下mailbox邮件箱里最新一封邮件内容。当前需求中可以作为函数参数的是：1.查看谁的邮件。"}, 
    {"role": "assistant", "content": "请帮我编写一个python函数，用于查看我在mailbox邮件箱中最新一封邮件的信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数里；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"}], 
"stage2": [
    {"role": "user", "content": "请帮我编写一个python函数，用于查看我在mailbox邮件箱中最新一封邮件的信息，函数要求如下：                 1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；                 2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；                 3.请将全部功能封装在一个函数里；                 4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"}, 
    {"role": "assistant", "content": "def get_latest_email(userId):\n    \"\

In [144]:
code = json.loads(json_str)['stage2'][1]['content']
code

'def get_latest_email(userId):\n    """\n    获得最新的一封邮件\n    参数:\n    user_id: 要检索邮件的用户的ID。默认值是\'me\'，数据格式为字符串类型。\n\n    返回:\n    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：\n    \'from_userid\': 发件人ID。\n    \'date\': 邮件的发送日期。\n    \'subject\': 邮件的主题。\n    \'snippet\': 邮件的摘要（前100个字符）。\n    """\n    mysql_pw="gpt"\n    \n    connection = pymysql.connect(\n    host=\'localhost\',  # 数据库地址\n    user=\'gpt\',  # 数据库用户名\n    passwd=mysql_pw,  # 数据库密码\n    db=\'my_mail\',  # 数据库名\n    charset=\'utf8\'  # 字符集选择utf8   \n    )\n    sql_query=f"SELECT * FROM mailbox WHERE userid = \'{userId}\' ORDER BY date DESC LIMIT 1";\n\n    try:\n        with connection.cursor() as cursor:\n            sql = sql_query\n            cursor.execute(sql)\n            results = cursor.fetchall()\n    finally:\n        cursor.close()\n    \n    column_names = [desc[0] for desc in cursor.description]\n\n    # 使用results和column_names创建DataFrame\n    df = pd.DataFrame(results, columns=column_names)\n        \n    return df.to

In [130]:
python_code=extract_code(code)
print(python_code)

def get_latest_email(userId):
    """
    获得最新的一封邮件
    参数:
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    """
    mysql_pw="gpt"
    
    connection = pymysql.connect(
    host='localhost',  # 数据库地址
    user='gpt',  # 数据库用户名
    passwd=mysql_pw,  # 数据库密码
    db='my_mail',  # 数据库名
    charset='utf8'  # 字符集选择utf8   
    )
    sql_query=f"SELECT * FROM mailbox WHERE userid = '{userId}' ORDER BY date DESC LIMIT 1";

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        cursor.close()
    
    column_names = [desc[0] for desc in cursor.description]

    # 使用results和column_names创建DataFrame
    df = pd.DataFrame(results, columns=column_names)
        
    return df.to_json(orient = "records")


In [134]:
exec(python_code)

In [136]:
print(get_latest_email("me"))

[{"userid":"me","from_userid":"\u897f\u95e8\u5439\u96ea","date":"2024-06-07","subject":"\u5f00\u4f1a","snippet":"\u660e\u592910\u70b9\u5230\u6d77\u6dc0\u533a\u53f6\u9752\u5927\u53a65\u697c502\u5f00\u4f1a","status":"\u5df2\u8bfb","label":"\u5f00\u4f1a"}]


我们发现根据推理链，大模型也能正确修复我们的代码

## 2. 基于大模型的自动Debug的函数

### 2.1 自动Debug函数构建

- 先定义一个审查函数

In [147]:
def prompt_modified(function_name, system_content='./data/推理链.md', model="glm-4", g=globals()):
    """
    邮箱项目的外部函数审查函数，用于审查外部函数创建流程提示是否正确以及最终创建的代码是否正确
    :param function_name: 必要参数，字符串类型，表示审查对象名称；
    :param system_content: 可选参数，默认取值为字符串推理链修改.md，表示此时审查函数外部挂载文档名称，需要是markdwon格式文档；
    :param model: 可选参数，表示调用的Chat模型，默认选glm-4；
    :param g: 可选参数，表示extract_function_code函数作用域，默认为globals()，即在当前操作空间全域内生效；
    :return：审查结束后新创建的函数名称
    """
    print("正在执行审查函数，审查对象：%s" % function_name)
    
    with open("./data/my_mail数据字典.md", 'r', encoding='utf-8') as f:
        data_content = f.read()
        
    with open(system_content, 'r', encoding='utf-8') as f:
        md_content = f.read()
        
    # 读取原函数全部提示内容
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
        msg = json.load(f)
    
    # 将其保存为字符串
    msg_str = json.dumps(msg)
    
    # 进行审查
    response = client.chat.completions.create(
                        model=model,
                        messages=[
                            {"role": "system", "content": data_content},
                            {"role": "system", "content": md_content},
                            {"role": "user", "content": '以下是一个错误的邮箱项目的推理链，请你按照要求对其进行修改：%s' % msg_str}
                          ]
        )
    
    modified_result = response.choices[0].message.content
    
    def extract_json(s):
        pattern = r'```[jJ][sS][oO][nN]\s*({.*?})\s*```'
        match = re.search(pattern, s, re.DOTALL)
        if match:
            return match.group(1)
        else:
            return s
    
    modified_json = extract_json(modified_result)
    
    # 提取函数源码
    code = json.loads(modified_json)['stage2'][1]['content']
    
    # 提取函数名
    match = re.search(r'def (\w+)', code)
    function_name = match.group(1)
    
    print("审查结束，新的函数名称为：%s。\n正在运行该函数定义过程，并保存函数源码与prompt" % function_name)
    
    exec(code, g)
    
    # 在untested文件夹内创建函数同名文件夹
    directory = './functions/untested functions/%s' % function_name
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    # 写入函数
    with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), 'w', encoding='utf-8') as f:
        f.write(code)
        
    # 写入提示
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'w') as f:
        json.dump(json.loads(modified_json), f)
    
    print('新函数提示示例保存在./functions/untested functions/%s/%s_prompt.json文件中' % (function_name, function_name))
    print("%s函数已在当前操作空间定义，可以进行效果测试" % function_name)
    
    return function_name

In [157]:
prompt_modified(function_name='get_latest_email', model="glm-4", g=globals())

正在执行审查函数，审查对象：get_latest_email
审查结束，新的函数名称为：get_latest_email。
正在运行该函数定义过程，并保存函数源码与prompt
新函数提示示例保存在./functions/untested functions/get_latest_email/get_latest_email_prompt.json文件中
get_latest_email函数已在当前操作空间定义，可以进行效果测试


'get_latest_email'

In [158]:
get_latest_email?

Signature: get_latest_email(userId='me')
Docstring:
获取最新的一封邮件
参数:
userId: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。

返回:
一个字典，表示一封邮件。字典包含以下键：
'from_userid': 发件人ID。
'date': 邮件的发送日期。
'subject': 邮件的主题。
'snippet': 邮件的摘要（前100个字符）。
File:      Dynamically generated function. No source code available.
Type:      function

In [159]:
get_latest_email()

'[{"userid":"me","from_userid":"\\u897f\\u95e8\\u5439\\u96ea","date":"2024-06-07","subject":"\\u5f00\\u4f1a","snippet":"\\u660e\\u592910\\u70b9\\u5230\\u6d77\\u6dc0\\u533a\\u53f6\\u9752\\u5927\\u53a65\\u697c502\\u5f00\\u4f1a","status":"\\u5df2\\u8bfb","label":"\\u5f00\\u4f1a"}]'

结论：
审查函数能让大模型输出正确的结果

- 构建自动Debug的函数

In [41]:
def extract_function_code(s, detail=0, tested=False, g=globals()):
    """
    函数提取函数，同时执行函数内容，可以选择打印函数信息，并选择代码保存的地址
    """
    def extract_code(s):
        """
        如果输入的字符串s是一个包含Python代码的Markdown格式字符串，提取出代码部分。
        否则，返回原字符串。

        参数:
        s: 输入的字符串。

        返回:
        提取出的代码部分，或原字符串。
        """
        # 判断字符串是否是Markdown格式
        if '```python' in s or 'Python' in s or'PYTHON' in s:
            # 找到代码块的开始和结束位置
            code_start = s.find('def')
            code_end = s.find('```\n', code_start)
            # 提取代码部分
            code = s[code_start:code_end]
        else:
            # 如果字符串不是Markdown格式，返回原字符串
            code = s

        return code
    
    # 提取代码字符串
    code = extract_code(s)
    
    # 提取函数名称
    match = re.search(r'def (\w+)', code)
    function_name = match.group(1)
    
    # 在untested文件夹内创建函数同名文件夹
    directory = './functions/untested functions/%s' % function_name
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # 将函数写入本地
    if tested == False:
        with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), 'w', encoding='utf-8') as f:
            f.write(code)
    else:
        # 调用remove_to_test函数将函数文件夹转移至tested文件夹内
        remove_to_tested(function_name)
        with open('./functions/tested functions/%s/%s_module.py' % (function_name, function_name), 'w', encoding='utf-8') as f:
            f.write(code)
    
    # 执行该函数
    try:
        exec(code, g)
    except Exception as e:
        print("An error occurred while executing the code:")
        print(e)
    
    # 打印函数名称
    if detail == 0:
        print("The function name is:%s" % function_name)
    
    if detail == 1:
        if tested == False:
            with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), 'r', encoding='utf-8') as f:
                content = f.read()
        else:
            with open('./functions/tested functions/%s/%s_module.py' % (function_name, function_name), 'r', encoding='utf-8') as f:   
                content = f.read()
                
        print(content)
        
    return function_name

In [42]:
def show_functions(tested=False, if_print=False):
    """
    打印tested或untested文件夹内全部函数
    """
    current_directory = os.getcwd()
    if tested == False:
        directory = current_directory + '/functions/untested functions'
    else:
        directory = current_directory + '/functions/tested functions'
    files_and_directories = os.listdir(directory)
    # 过滤结果，只保留.py文件和非__pycache__文件夹
    files_and_directories = files_and_directories = [name for name in files_and_directories if (os.path.splitext(name)[1] == '.py' or os.path.isdir(os.path.join(directory, name))) and name != ".ipynb_checkpoints"]
    
    if if_print != False:
        for name in files_and_directories:
            print(name)
    
    return files_and_directories

In [43]:
def code_generate(req, few_shot='all', model='glm-4', g=globals(), detail=0):
    """
    Function calling外部函数自动创建函数，可以根据用户的需求，直接将其翻译为Chat模型可以直接调用的外部函数代码。
    :param req: 必要参数，字符串类型，表示输入的用户需求；
    :param few_shot: 可选参数，默认取值为字符串all，用于描述Few-shot提示示例的选取方案，当输入字符串all时，则代表提取当前外部函数库中全部测试过的函数作为Few-shot；\
    而如果输入的是一个包含了多个函数名称的list，则表示使用这些函数作为Few-shot。
    :param model: 可选参数，表示调用的Chat模型，默认选取glm-4；
    :param g: 可选参数，表示extract_function_code函数作用域，默认为globals()，即在当前操作空间全域内生效；
    :param detail: 可选参数，默认取值为0，还可以取值为1，表示extract_function_code函数打印新创建的外部函数细节；
    :return：新创建的函数名称。需要注意的是，在函数创建时，该函数也会在当前操作空间被定义，后续可以直接调用；
    """
    
    # 提取提示示例的函数名称
    if few_shot == 'all':
        few_shot_functions_name = show_functions(tested=True)
    elif type(few_shot) == list:
        few_shot_functions_name = few_shot
    # few_shot_functions = [globals()[name] for name in few_shot_functions_name]
    
    # 读取各阶段系统提示
    with open('./functions/tested functions/system_messages.json', 'r') as f:
        system_messages = json.load(f)
        
    # 各阶段提示message对象
    few_shot_messages_CM = []
    few_shot_messages_CD = []
    
    few_shot_messages = []
    
    # 先保存第一条消息，也就是system message
    few_shot_messages_CD += system_messages["system_message_CD"]
    few_shot_messages_CM += system_messages["system_message_CM"]
    few_shot_messages += system_messages["system_message"]

    # 创建不同阶段提示message
    for function_name in few_shot_functions_name:
        with open('./functions/tested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
            msg = json.load(f)
        few_shot_messages_CD += msg["stage1_CD"]
        few_shot_messages_CM += msg["stage1_CM"]
        few_shot_messages += msg['stage2']
        
    # 读取用户需求，作为第一阶段CD环节User content
    new_req_CD_input = req
    few_shot_messages_CD.append({"role": "user", "content": new_req_CD_input})
    
    print('第一阶段CD环节提示创建完毕，正在进行CD提示...')
    
    # 第一阶段CD环节Chat模型调用过程
    response = client.chat.completions.create(
                        model=model,
                        messages=few_shot_messages_CD)
    new_req_pi = response.choices[0].message.content
    
    #print("打印内容1："+new_req_pi)
    
    print('第一阶段CD环节提示完毕')
    
    # 第一阶段CM环节Messages创建
    new_req_CM_input = new_req_CD_input + new_req_pi
    few_shot_messages_CM.append({"role": "user", "content": new_req_CM_input})
    
    print('第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...')
    # 第一阶段CM环节Chat模型调用过程
    response = client.chat.completions.create(
                        model=model,
                        messages=few_shot_messages_CM)
    new_req_description = response.choices[0].message.content
    
    #print("打印内容2："+new_req_description)
    #return "test"

    
    print('第一阶段CM环节提示完毕')
    
    # 第二阶段Messages创建过程
    few_shot_messages.append({"role": "user", "content": new_req_description})
    
    print('第二阶段提示创建完毕，正在进行第二阶段提示...')
    
    # 第二阶段Chat模型调用过程
    response = client.chat.completions.create(
                        model=model,
                        messages=few_shot_messages)
    new_req_function = response.choices[0].message.content
    
    
    print('第二阶段提示完毕，准备运行函数并编写提示示例')
    
    # 提取函数并运行，创建函数名称对象，统一都写入untested文件夹内
    function_name = extract_function_code(s=new_req_function, detail=detail, g=g)
    
    print('新函数保存在./functions/untested functions/%s/%s_module.py文件中' % (function_name, function_name))
    
    # 创建该函数提示示例
    new_req_messages_CD = [
                          {"role": "user", "content": new_req_CD_input},
                          {"role": "assistant", "content": new_req_pi}
                         ]
    new_req_messages_CM = [
                          {"role": "user", "content": new_req_CM_input},
                          {"role": "assistant", "content":new_req_description}
                         ]
    
    with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), encoding='utf-8') as f:
        new_req_function = f.read()
    
    new_req_messages = [
                       {"role": "user", "content": new_req_description},
                       {"role": "assistant", "content":new_req_function}
                      ] 
    
    new_req_prompt = {
                     "stage1_CD": new_req_messages_CD,
                     "stage1_CM": new_req_messages_CM,
                     "stage2": new_req_messages
                    }   
    
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'w') as f:
        json.dump(new_req_prompt, f)
        
    print('新函数提示示例保存在./functions/untested functions/%s/%s_prompt.json文件中' % (function_name, function_name))
    print('done')
    return function_name

In [161]:
def remove_to_tested(function_name):
    """
    将函数同名文件夹由untested文件夹转移至tested文件夹内。\
    完成转移则说明函数通过测试，可以使用。
    """
    
    # 将函数代码写入glmLesson.py文件中
    with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), encoding='utf-8') as f:
        function_code = f.read()
    
    # 源文件夹路径
    src_dir = './functions/untested functions/%s' % function_name

    # 目标文件夹路径
    dst_dir = './functions/tested functions/%s' % function_name
    
    # 移动文件夹
    shutil.move(src_dir, dst_dir)

In [35]:
def auto_functions(functions_list):
    """
    Chat模型的functions参数编写函数
    :param functions_list: 包含一个或者多个函数对象的列表；
    :return：满足Chat模型functions参数要求的functions对象
    """
    def functions_generate(functions_list):
        # 创建空列表，用于保存每个函数的描述字典
        functions = []
        # 对每个外部函数进行循环
        for function in functions_list:
            # 读取函数对象的函数说明
            function_description = inspect.getdoc(function)
            # 读取函数的函数名字符串
            function_name = function.__name__
            ## 提示词 one-shot
            system_prompt1="""输出结果必须是一个JSON格式的字典,举个例子：
            [{'type': 'function', 'function': {'name': 'sunwukong_function', 'description': '该函数定义了数据集计算过程，接收一个数据表字符串参数，并返回一个JSON格式的Dataframe类型对象。', 'parameters': {'type': 'object', 'properties': {'data': {'type': 'string', 'description': '带入计算的数据表，用字符串进行表示'}}, 'required': ['data'], 'additionalProperties': False}}}]
            """
            system_prompt2 = '以下是某的函数说明：%s,输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_description
            user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
                           1.字典总共有三个键值对；\
                           2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
                           3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
                           4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
                           5.输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_name

            response = client.chat.completions.create(
                              model="glm-4",
                              messages=[
                                {"role": "system", "content": system_prompt1},
                                {"role": "system", "content": system_prompt2},
                                {"role": "user", "content": user_prompt}
                              ]
                            )
            json_str=response.choices[0].message.content.replace("```json","").replace("```","")
            json_function_description=json.loads(json_str)
            json_str={"type": "function","function":json_function_description}
            functions.append(json_str)
        return functions
    ## 最大可以尝试4次
    max_attempts = 4
    attempts = 0

    while attempts < max_attempts:
        try:
            functions = functions_generate(functions_list)
            break  # 如果代码成功执行，跳出循环
        except Exception as e:
            attempts += 1  # 增加尝试次数
            print("发生错误：", e)
            if attempts == max_attempts:
                print("已达到最大尝试次数，程序终止。")
                raise  # 重新引发最后一个异常
            else:
                print("正在重新运行...")
    return functions

In [36]:
def run_conversation(messages, functions_list=None, model="glm-4"):
    """
    能够自动执行外部函数调用的对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为glm-4
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response.choices[0].message
        final_response = response_message.content
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        tools = auto_functions(functions_list)
        
        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}
        print(available_functions)

        # 第一次调用大模型
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        tools=tools,
                        tool_choice="auto", )
        response_message = response.choices[0].message
        
       


        tool_calls = response_message.tool_calls

        if tool_calls:

            #messages.append(response.choices[0].message)
            ## GLM-4 Function Call的使用方式
            messages.append(response.choices[0].message.model_dump())
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                ## 真正执行外部函数的就是这儿的代码
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "role": "tool",
                        "content": function_response,
                        "tool_call_id": tool_call.id,
                    }
                ) 
            ## 第二次调用模型
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools
            ) 
            # 获取最终结果
            print(second_response.choices[0].message)
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
                
    return final_response

In [44]:
def function_test(function_name, req, few_shot, model="glm-4", g=globals()):

    def test_messages(ueser_content):
        messages = [{"role": "system", "content": data_content},
                     {"role": "system", "content": "我的邮箱号叫me"},
                    {"role": "user", "content": ueser_content}]
        return messages
            
    messages = test_messages(req)
    
    new_function = globals()[function_name]
    functions_list = [new_function]
    
    print("根据既定用户需求req进行%s函数功能测试，请确保当该函数已经在当前操作空间定义..." % function_name)
    
    # 有可能在run_conversation环节报错
    # 若没报错，则运行：
    try:
        ## 直接调用看当前函数给出来的结果是否正确
        ## 有两种可能：1，代码能运行 2. 代码不能运行
        final_response = run_conversation(messages=messages, functions_list=functions_list, model=model)
        print("当前函数运行结果：'%s'" % final_response)
        ## 如果代码能运行看一下是否能满足需求
        feedback = input("函数功能是否满足要求 (yes/no)? ")
        if feedback.lower() == 'yes':
            print("函数功能通过测试，正在将函数写入tested文件夹")
            ## 如果满足就通过测试
            remove_to_tested(function_name)
            print('done')
        else:
            ## 如果不满足需求
            next_step = input("函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？")
            if next_step == '1':
                print("准备再次测试...")
                ## 再次运行
                function_test(function_name, req, few_shot)
            else:
                ## 进行Debug，选择方案
                solution = input("请选择debug方案：\n1.再次执行函数创建流程，并测试结果；\n2.执行审查函数\
                \n3.重新输入用户需求；\n4.退出程序，进行手动尝试")
                if solution == '1':
                    # 再次运行函数创建过程
                    print("好的，正在尝试再次创建函数，请稍等...")
                    few_shot_str = input("准备再次测试，请问是1.采用此前Few-shot方案，还是2.带入全部函数示例进行Few-shot？")
                    if few_shot_str == '1':
                        function_name = code_generate(req=req, few_shot=few_shot, model=model, g=g)
                    else:
                        function_name = code_generate(req=req, few_shot='all', model=model, g=g)
                    ## 构建完了以后继续运行
                    function_test(function_name=function_name, req=req, few_shot=few_shot, g=g)
                elif solution == '2':
                    # 执行审查函数
                    print("好的，执行审查函数，请稍等...")
                    function_name = prompt_modified(function_name=function_name, model="gpt-3.5-turbo-16k-0613", g=g)
                    # 接下来带入进行测试
                    print("新函数已创建，接下来带入进行测试...")
                    function_test(function_name=function_name, req=req, few_shot=few_shot, g=g)
                elif solution == '3':
                    ## 用户重新输入需求，也就意味着重新
                    new_req = input("好的，请再次输入用户需求，请注意，用户需求描述方法将极大程度影响最终函数创建结果。")
                    few_shot_str = input("接下来如何运行代码创建函数？1.采用此前Few-shot方案；\n2.使用全部外部函数作为Few-shot")
                    if few_shot_str == '1':
                        function_name = code_generate(req=new_req, few_shot=few_shot, model=model, g=g)
                    else:
                        function_name = code_generate(req=new_req, few_shot='all', model=model, g=g)
                    function_test(function_name=function_name, req=new_req, few_shot=few_shot, g=g)
                elif solution == '4':
                    print("加油~")
        
    # run_conversation报错时则运行：
    except Exception as e:
        next_step = input("run_conversation无法正常运行，接下来是1.再次运行运行run_conversation，还是2.进入debug流程？")
        if next_step == '1':
            function_test(function_name, req, few_shot)
        else:
            solution = input("请选择debug方案：\n1.再次执行函数创建流程，并测试结果；\n2.执行审查函数\
            \n3.重新输入用户需求；\n4.退出程序，进行手动尝试")
            if solution == '1':
                # 再次运行函数创建过程
                print("好的，正在尝试再次创建函数，请稍等...")
                few_shot_str = input("准备再次测试，请问是1.采用此前Few-shot方案，还是2.带入全部函数示例进行Few-shot？")
                if few_shot_str == '1':
                    function_name = code_generate(req=req, few_shot=few_shot, model=model, g=g)
                else:
                    function_name = code_generate(req=req, few_shot='all', model=model, g=g)
                function_test(function_name=function_name, req=req, few_shot=few_shot, g=g)
            elif solution == '2':
                # 执行审查函数
                print("好的，执行审查函数，请稍等...")
                max_attempts = 3
                attempts = 0

                while attempts < max_attempts:
                    try:
                        function_name = prompt_modified(function_name=function_name, model="gpt-3.5-turbo-16k-0613", g=g)
                        break  # 如果代码成功执行，跳出循环
                    except Exception as e:
                        attempts += 1  # 增加尝试次数
                        print("发生错误：", e)
                        if attempts == max_attempts:
                            print("已达到最大尝试次数，程序终止。")
                            raise  # 重新引发最后一个异常
                        else:
                            print("正在重新运行审查程序...")
                # 接下来带入进行测试
                print("新函数已创建，接下来带入进行测试...")
                function_test(function_name=function_name, req=req, few_shot=few_shot, g=g)
            elif solution == '3':
                new_req = input("好的，请再次输入用户需求，请注意，用户需求描述方法将极大程度影响最终函数创建结果。")
                few_shot_str = input("接下来如何运行代码创建函数？1.采用此前Few-shot方案；\n2.使用全部外部函数作为Few-shot")
                if few_shot_str == '1':
                    function_name = code_generate(req=new_req, few_shot=few_shot, model=model, g=g)
                else:
                    function_name = code_generate(req=new_req, few_shot='all', model=model, g=g)
                function_test(function_name=function_name, req=new_req, few_shot=few_shot, g=g)
            elif solution == '4':
                print("加油~")

**测试**

准备了一个有问题的函数

In [167]:
def get_latest_email(userId):
    """
    获取最新的一封邮件
    参数:
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    """
    mysql_pw="gpt"
    
    connection = pymysql.connect(
    host='localhost',  # 数据库地址
    user='gpt',  # 数据库用户名
    passwd=mysql_pw,  # 数据库密码
    db='my_mail',  # 数据库名
    charset='utf8'  # 字符集选择utf8   
    )
    sql_query=f"SELECT * FROM mailbox WHERE userid = '{me}' ORDER BY date DESC LIMIT 1";

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        cursor.close()
    
    column_names = [desc[0] for desc in cursor.description]

    # 使用results和column_names创建DataFrame
    df = pd.DataFrame(results, columns=column_names)
        
    return df.to_json(orient = "records")  

In [168]:
function_test(function_name = 'get_latest_email', 
              req = '请帮我查下mailbox邮箱里最新一封邮件内容。', 
              few_shot = 'all')

根据既定用户需求req进行get_latest_email函数功能测试，请确保当该函数已经在当前操作空间定义...


run_conversation无法正常运行，接下来是1.再次运行运行run_conversation，还是2.进入debug流程？ 1


根据既定用户需求req进行get_latest_email函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'好的，我将连接到数据库，并查询mailbox数据表中最新的一封邮件内容。 

```python
import mysql.connector

# 连接到数据库
db = mysql.connector.connect(
    host='localhost',
    user='gpt',
    passwd='gpt',
    database='my_mail'
)

cursor = db.cursor()

# 查询最新一封邮件内容
query = "SELECT * FROM mailbox ORDER BY date DESC LIMIT 1"
cursor.execute(query)

# 获取查询结果
latest_mail = cursor.fetchone()

# 关闭数据库连接
cursor.close()
db.close()

# 显示最新一封邮件内容
latest_mail
```

执行以上代码，将会获取并显示最新一封邮件的详细内容。请确认我是否可以执行代码来进行查询。'


函数功能是否满足要求 (yes/no)?  no
函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数                
3.重新输入用户需求；
4.退出程序，进行手动尝试 2


好的，执行审查函数，请稍等...
正在执行审查函数，审查对象：get_latest_email
审查结束，新的函数名称为：get_latest_email。
正在运行该函数定义过程，并保存函数源码与prompt
新函数提示示例保存在./functions/untested functions/get_latest_email/get_latest_email_prompt.json文件中
get_latest_email函数已在当前操作空间定义，可以进行效果测试
新函数已创建，接下来带入进行测试...
根据既定用户需求req进行get_latest_email函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'最新一封邮件的内容如下：

发件人: 西门吹雪
收件人: me
日期: 2024-06-07
主题: 开会
邮件摘要: 明天10点到海淀区叶青大厦5楼502开会
邮件状态: 已读
邮件标签: 开会

请注意查收。'


函数功能是否满足要求 (yes/no)?  yes


函数功能通过测试，正在将函数写入tested文件夹
done


### 2.2 全自动流程代码编写

In [195]:
def req_auto_func(req, few_shot='all', model='glm-4', g=globals(), detail=0):
    function_name = code_generate(req, few_shot=few_shot, model=model, g=g, detail=detail)
    function_test(function_name=function_name, req=req, few_shot=few_shot, model=model, g=g)

## 3. 面向目标编程测试

- Q1.'请帮我看下我的邮箱里面，来自西门吹雪的邮件总共有几封。'

In [207]:
req_auto_func(req = '请帮我查下mailbox邮箱里，来自西门吹雪的邮件总共有几封。')

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
来自 西门吹雪 的邮件总共有 {"total_emails": 2} 封。
The function name is:count_emails_from_sender
新函数保存在./functions/untested functions/count_emails_from_sender/count_emails_from_sender_module.py文件中
新函数提示示例保存在./functions/untested functions/count_emails_from_sender/count_emails_from_sender_prompt.json文件中
done
根据既定用户需求req进行count_emails_from_sender函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'在mailbox邮箱里，来自西门吹雪的邮件总共有2封。'


函数功能是否满足要求 (yes/no)?  yes


函数功能通过测试，正在将函数写入tested文件夹
done


- Q2.'请帮我统计下我的邮箱里面总共有几封未读邮件'

In [206]:
req_auto_func(req = '请帮我统计下我的邮箱里面总共有几封未读邮件')

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
An error occurred while executing the code:
invalid syntax (<string>, line 41)
The function name is:count_unread_emails
新函数保存在./functions/untested functions/count_unread_emails/count_unread_emails_module.py文件中
新函数提示示例保存在./functions/untested functions/count_unread_emails/count_unread_emails_prompt.json文件中
done
根据既定用户需求req进行count_unread_emails函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'在您的邮箱中，您总共有2封未读邮件。'


函数功能是否满足要求 (yes/no)?  yes


函数功能通过测试，正在将函数写入tested文件夹
done


- Q3.'请帮我统计下我的邮箱里面总共有多少封内容包含Q2的邮件。'

In [215]:
req_auto_func(req = '请帮我统计下我的邮箱里面总共有多少封内容包含Q2的邮件。')

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:search_email_by_keyword
新函数保存在./functions/untested functions/search_email_by_keyword/search_email_by_keyword_module.py文件中
新函数提示示例保存在./functions/untested functions/search_email_by_keyword/search_email_by_keyword_prompt.json文件中
done
根据既定用户需求req进行search_email_by_keyword函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'为了统计邮箱中包含“Q2”内容的邮件数量，可以使用如下SQL查询：

```sql
SELECT COUNT(*) AS total_Q2_emails
FROM mailbox
WHERE userid = 'me' AND snippet LIKE '%Q2%';
```

这条查询语句的逻辑是：
- `SELECT COUNT(*) AS total_Q2_emails`: 计算符合条件的邮件数量，并将结果命名为 `total_Q2_emails`。
- `FROM mailbox`: 从 `mailbox` 表中查询数据。
- `WHERE userid = 'me' AND snippet LIKE '%Q2%'`: 只选择收件人为 `me` 且内容包含 `Q2` 的邮件。

可以通过执行该SQL查询来获得结果。如果需要我来帮你执行查询，请告诉我！'


函数功能是否满足要求 (yes/no)?  no
函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？ 1


准备再次测试...
根据既定用户需求req进行search_email_by_keyword函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'
```
import mysql.connector

# Connect to the database
db_connection = mysql.connector.connect(
    host='localhost',
    user='gpt',
    passwd='gpt',
    database='my_mail'
)

# Create a cursor object
cursor = db_connection.cursor()

# Define the query to count emails containing 'Q2' in the subject or snippet
query = """
SELECT COUNT(*) 
FROM mailbox 
WHERE userid = 'me' AND (subject LIKE '%Q2%' OR snippet LIKE '%Q2%')
"""

# Execute the query
cursor.execute(query)

# Fetch the result
result = cursor.fetchone()

# Close the cursor and connection
cursor.close()
db_connection.close()

# Output the result
result[0]
```
`Result:` 
```
---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
Cell In[1], line 1
----> 1 import mysql.connector
      3 # Connect to the database
      4 db_connection = mysql.connecto

函数功能是否满足要求 (yes/no)?  no
函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？ 1


准备再次测试...
根据既定用户需求req进行search_email_by_keyword函数功能测试，请确保当该函数已经在当前操作空间定义...
当前函数运行结果：'相关邮件有1封。具体信息如下：

- 邮件发送者：陆小凤

- 发送日期：2024-06-03

- 主题：2024年Q2规划

- 邮件摘要：李先生，您好，我是陆小凤，这是我给您发的邮件，里面是Q2规划：聚焦核心业务，优化流程，提升效率。加强团队协作，培养人才。探索新市场，拓宽业务领域。确保目标达成，实现增长。详细见邮件附件

- 邮件状态：未读

- 邮件标签：规划'


函数功能是否满足要求 (yes/no)?  yes


函数功能通过测试，正在将函数写入tested文件夹
done


至此，我们就完成了一个高度自动化的编程函数。同时，该流程也是目前业内大语言自动编程最完善、也是单函数编程功能最强的流程之一。接下来我们的编程共工作也将主要依赖该函数完成。

## 4 大模型进阶复杂任务

### 4.1 大模型编程的瓶颈

&emsp;&emsp;在完成了全自动代码编写和审查函数之后，我们已经能够引导大模型完成大量代码开发工作了。但是受到大模型本身理解能力和编程能力限制，在面对一些复杂需求或者处理一些大模型本身不太容易理解的问题时（如处理时间类问题），这套流程仍然无法稳定高效的完成相关函数的代码编写工作。当然，问题越难，大模型编写代码的准确率就越低。例如面对如下需求：请帮我看下我的邮箱里面，最早的一封来自西门吹雪的邮件是什么时候，以及邮件的核心内容是什么？在调用此前定义的req_auto_func函数时，经过若干轮的调试和debug，仍然无法获得正确的结果：

In [ ]:
req_auto_func(req = '请帮我看下我的邮箱里面，最早的一封来自西门吹雪的邮件是什么时候，以及邮件的核心内容是什么？')

> 需要注意的是，由于大模型编程本身存在一定的不稳定性，因此这里列举的两个需求示例在某些情况下也是能够被编程成功的，但相比于其他简单需求，大模型对于复杂需求的编程准确性明显大幅下降。

&emsp;&emsp;经过上述验证不难发现，尽管当前的大模型编程流程已经添加了非常多的提高稳定性的措施（诸如多段提示、提示流程审查和代码审查等），但对于复杂的编程任务，大模型编程的准确率仍然很低。面对这一问题，有什么解决方案呢？需要注意的是，对于大模型工程师来说，如何“用好”大模型始终是岗位核心竞争力，而自然语言编程又是大模型非常重要的应用领域，因此，更进一步的思考和探讨如何进一步提高大模型对复杂问题的编程能力，是非常有必要的。

### 4.2 编程性能瓶颈根源与解决方案

- 编程性能瓶颈根源

&emsp;&emsp;其实大模型对复杂问题的编程能力不足，和大模型对于创作长文本能力不足的原因是一样的——都是因为模型本身并不具备“长期记忆”，而这会导致大模型无法一直围绕某个恒定的目标进行长段的文本生成，需要注意的是，对于大模型来说，代码编写本质上就是按照某种特定格式进行文本创作，本质上并没有区别，而面对复杂需求，往往意味着为了实现该需求需要编写长段的代码，因此受限于大模型编写长段代码能力不足，对于复杂需求的代码编写准确性也随之降低。

- 提升长段代码编程稳定性策略

&emsp;&emsp;而要如何解决这个问题呢？首先需要明确的是，此时大模型所表现出来的问题，并不是需求理解不足，而是在充分理解需求的情况下，也无法编写长段代码来满足需求，也就是说大模型在此前的LtM提示流程下是能够充分理解用户需求的，也能够顺利的辨别需求背后的变量（作为函数参数），问题在于实现该需求层面代码本身编写能力不足。因此我们解决问题的重点是如何提高模型的编程能力（而不是通过提示工程来进一步提升模型理解能力。）。

&emsp;&emsp;那么应该如何提升大模型的编写长段代码的稳定性呢？和编写长文本类似，最简单有效的方法就是先列提纲，然后分段编写，最后进行合并，即通过分段完成的方式，来提高模型长文本（长段代码）编写的表现。当然，编程任务还有其特殊之处，例如分段编写的代码需要变量一致，并且前后代码功能需要很好的衔接，此外我们已经编写了非常多辅助代码编写的函数，还需要考虑尽可能借助这些辅助函数来提高开发效率等。而我们也已经积累了较为丰富的大语言模型相关开发流程的设计经验，并且也具备了一定的“调教”大模型的经验，接下来我们在“分段编写再合并”这一基本思路指导下，先尝试头脑风暴，尽可能列举理论上可行的实现方案，然后再选择其一进行落地代码实现。

- 第一阶段：复杂需求拆解

&emsp;&emsp;既然是要分段编写代码再进行合并，首先就需要先对原始的复杂需求进行拆解。这个拆解的过程就非常类似于CoT思考链——一步步思考并解决问题。例如对于需求：请将我邮箱里面未读邮件中的有关开会邀请的邮件添加上会议预定的标签。就可以拆分为步，第一步是找到所有未读邮件，第二步是在未读邮件中找到有关会议邀请的邮件，并将其添加会议预定的标签。不难看出，这一拆分过程本身并不复杂，甚至相比CoT，围绕当前邮件项目的复杂任务拆分流程会更加具备确定性，因此也会更加简单。

&emsp;&emsp;这里所谓的拆分任务的确定性，主要是因为拆分得到的子任务往往是一些最小功能单位，比如查找邮件、给邮件打标签等。而一旦拆分的子任务属于较为确定性的一类任务，那么这个拆分任务这一过程本身，其实就是一类特殊且简单的拆分任务。而要引导大语言模型完成特殊且较为简单的任务，最好的方式就是输入一些项目背景信息（挂载外部文档）并且提供一些拆解的示例作为Few-shot。

&emsp;&emsp;这里需要注意的是，在进行子任务时，必须强调子任务需要是能够简单实现的子任务，否则模型在进行子任务拆分时甚至会认为第一步是打开邮箱，而另一方面，只有拆分的子任务是能够简单快速实现的任务，才能尽可能确保子任务对应的代码编写的准确性。而根据我们对大模型性能的了解，这方面信息的顺利传递，不能只通过Few-shot，还必须依靠编写一个外部文档进行项目说明。此外需要注意的是，这里如果找不到合适的任务拆分示例作为Few-shot，也可以在充分描述背景之后让大模型帮我们进行编写。

&emsp;&emsp;而当我们已经完成任务拆分之后，接下来就是考虑如何围绕子任务进行代码编写，以及各部分代码如何拼接等问题了。这些工作我们统一称为复杂任务编程的第二阶段，第二个阶段的实现方法可以有很多种，我们这里依次对其进行介绍：

- 第二阶段方案一：单独创建复杂需求函数

&emsp;&emsp;首先最容易想到的方法，就是在基础已有的基础功能函数基础之上再架设一层高层函数，用于组合基础函数功能，来实现更加复杂的需求。这里所谓的基础功能函数，指的是可以借助code_generate函数进行准确高效编写的基础功能函数，例如此前我们创建的查看未读邮件、发送邮件、邮件汇总等功能函数，而从任务拆分角度来看，这些基础函数就是专门用于去实现子任务的相关函数。而在面对复杂需求时，我们可以围绕拆分出来的子任务先分别编写对应的功能实现函数，然后再编写一个可以调用基础函数的高层函数，用于将基础函数的输出结果进行整合，并最终完成原始复杂任务的处理。

例如对于需求：请将我邮箱里面未读邮件中的有关会议邀请的邮件添加上会议预定的标签。就可以拆分为步，第一步是找到所有未读邮件，第二步是在未读邮件中找到有关会议邀请的邮件，并将其添加会议预定的标签。

&emsp;&emsp;例如，对于当前需求“请将我邮箱里面未读邮件中的有关会议邀请的邮件添加上会议预定的标签”（以下简称为复杂需求或者当前需求）来说，我们已经将其拆分为了两个子任务，分别是获取未读邮件列表（以下简称为子任务1），以及从未读邮件中挑选会议相关邮件，并为其添加会议预定标签（以下简称为子任务2）。这两个子任务可以分别借助此前定义好的code_generate函数+function_test函数进行创建，同时需要注意在创建完成子任务2的函数（以下简称为函数2）时，需要确保其输入的参数格式和完成子任务1的函数（以下简称为函数1）的输出结果一致。而当我们创建完函数1和函数2之后，接下来即可定义一个外部函数，用于串联函数1和函数2来进行运行，即可获得最终结果。

&emsp;&emsp;不难发现，方案一的这种分层开发的思路其实非常像传统技术团队的研发策略，这种策略便于分工、易于管理，并且有非常明显的阶段性产出，同时如果是人工进行代码编写，也能非常便捷的维持函数1的输出结果和函数2的参数一致。并且，这种开发方案的优势在于代码利用率很高，试想一下，伴随着基础函数库的函数功能不断丰富，会有越来越多的复杂需求或许并不需要单独编写子任务函数，而可以直接从已有的函数库中挑选函数来组合运行。例如，如果现在的需求是在未读邮件中寻找会议相关邮件，并将邮件内容编写为附件本地保存，则在分解任务时，分解得到的第一个子任务——列举未读邮件列表，就可以直接从利用函数1来完成。总的来说，方案一是一种易于管理且代码利用率极高的一个方案。

&emsp;&emsp;但是，该方案也存在一些问题，其中最核心的问题就在于现在是引导大模型进行代码开发，而不是引导人进行代码开发。这其中会存在较为本质的区别，对于程序员来说，能够非常简单的的统一一系列函数的输出和输入格式，但如果是自然语言编程，这一点就很难做到，我们很难向大模型传递“函数1的输出结果要用于函数2，因此函数1需要输出每一封未读邮件的邮件ID，否则函数2就无法定位具体的邮件”。因此，在实际执行方案一时，哪怕函数1和函数2都能顺利编写代码，但二者也很难顺利衔接到一起来进行运行。

&emsp;&emsp;其实对于大多数初级大模型工程师来说，都习惯以人的思维思考大模型的运行状态，而实际上，要设计一套引导大模型来执行任务的流程，就必须优先考虑到大模型和人的区别，很多看起来对人类程序员非常友好的流程，却不一定适用于引导大模型来完成。一个为大模型设计的任务流程需要充分发挥大模型的优势，而不能先入为主的优先从人的角度来考虑问题。

&emsp;&emsp;尽管大模型在函数功能的语义理解层面不如人类程序员，但大模型却有自己的得天独厚的优势——能够不知疲倦的反复高效的创建代码，并且能够对自己编写的代码进行自动审查。因此我们完全可以利用这点，来创建一个更适合大模型来执行的复杂需求函数编写流程。因此，接下来的两个方案不再考虑拆分创建个函数再组合运行的思路，而是考虑采用一个函数来满足复杂需求，并且充分借助大模型能够进行自我审查的特性，来提高这一个函数对需求满足的准确性。

- 第二阶段方案二：创建嵌套函数

&emsp;&emsp;而如何在一个函数内完成复杂需求功能设计，首先能想到的就是创建嵌套函数，即将方案一中的函数1作为内部函数，函数2作为外部函数来创建一个嵌套函数，并由此来满足原始需求。具体执行流程如下：首先我们先单独引导大模型编写函数1和函数2的需求，并且在提示中规定函数1的输出将作为函数2的输入，然后先创建函数2，在创建完函数2的代码之后，再将函数2的参数中原本用于表示函数1输出的参数，改为函数1的参数，然后再将函数2的代码作为提示，并且在提示中说明函数1和2的功能，以及函数1是函数2的内部函数等信息，以此引导大模型创建函数1。此时由于模型是在知道函数1和2之间的关系、以及函数2的代码基础之上创建的函数1，便能够自动判断需要将未读邮件的邮件ID作为函数1的输出，才能让函数2顺利的为某些函数打上标签。在创建完函数1和函数2之后，再将其进行合并（函数1作为函数2的内部函数），并输出最终结果。

&emsp;&emsp;很明显，该流程能够非常好的借助大模型本身的能力，解决此前所面临的函数1和2无法很好的协调输入和输入的问题，同时由于一个复杂需求一个函数，因此也不存在方案一中由于大模型语义理解不稳定性造成潜在风险。总的来说，相比方案一，方案二可以说是能够更好的发挥大模型本身的能力。但是，通过仔细观察上述方案流程不难发现，方案二最核心的问题就在于需要反复给与大模型非常多段的提示（例如函数2的参数修改、函数1和2的关系等），才能很好的引导大模型完成相关工作。尽管我们也可以通过创建某个函数来流程化的批量执行这些提示，但这个流程和此前我们创建的用于编写基础函数的流程差距太大，因此并不能最大程度借鉴此前流程，从而导致需要耗费大量的人工成本、且整体效率较低。

&emsp;&emsp;介于此，我们需要在方案二的基础之上，进一步优化整个流程，即在这个复杂函数自动编写流程中，不仅需要扬长避短发挥大模型特性，同时也需要尽可能做到简洁优雅，需要尽可能借用此前基础函数的开发流程，并且保证一个较高的准确性。

- 第二阶段方案三：分别创建函数再统一代码

&emsp;&emsp;而最后一个方案则是在前两个方案基础之上进行优化和迭代的最终版方案，该方案非常不符合人类程序员的一般开发思路，但却是非常适合大语言模型来执行的开发流程，并且能够非常好的复用此前的基础函数的提示和审查流程，从而极大程度提升开发效率。

&emsp;&emsp;方案三执行流程如下，首先和方案一一样，都是拆分两个子任务，并且分别编写两个函数以满足这两个子任务需求。并且在创建函数1和2时也是采用code_generate+function_test来进行代码编写和审查，但这里需要注意，我们在创建这两个函数时，会先创建函数1，然后将函数的输出结果作为函数2创建时的需求的一部分（注意，这里只涉及需求的拼接，不涉及提示流程的修改）。并且，我们在创建函数1和函数2时，只要求代码通顺即可，并不要求两个函数能够拼凑成一个完整的链路，即不要求函数1的输出结果是函数2的输入。此时根据我们对大模型编程能力的判断，大概率函数1和2能够分别顺利运行，但无法组成一个完整的链路。但没有关系，接下来我们再次调用code_generate+function_test流程来围绕原始需求进行函数编写（以下将该函数称为原始需求函数），并且在这个过程中同时将函数1和2的源码作为system_message带入这个编写过程，此时，大模型有函数1和2的代码作为参考，即可非常顺利的完成原始需求函数编写。需要注意的是，原始需求函数并不是函数1和函数2的拼接，而是在参考了函数1和2的代码基础之上，重新编写的一个独立且完整的函数，因此哪怕函数1和2本身并不能很好的进行衔接，也不会影响原始需求函数的代码正确性。并且，此时有了函数1和2的代码作为参考，大模型围绕复杂需求进行编程的准确性也将大幅提升。更重要的是，这个过程基本是完全复用code_generate+function_test流程，无需进行更复杂大模型提示即可完成，开发效率很高。

### 4.3 复杂任务的编程落地

- Stage 1.复杂需求拆解

&emsp;&emsp;首先，我们需要创建一个专门用于复杂问题拆解的模型。根据此前的分析，该模型同时需要外部挂载文档以说明项目背景和复杂任务拆解的目的，同时还需要一些复杂任务拆解的示例作为Few-shot。

In [29]:
with open('./data/复杂任务拆解.md', 'r', encoding='utf-8') as f:
    md_content_plus = f.read()
    
print(md_content_plus)

1.什么是邮箱项目？邮件项目本身由一系列的程序组成，核心功能是根据用户自然语言描述进行相应的邮箱操作，如查阅邮件、收发邮件等。该场景是我们自己构建的一个场景。
主要实现的是对我们my_mail数据库里的数据进行增删查改。

2.什么是任务拆解？在邮箱项目中，任务拆解指的是将用户需求拆解为若干个最小子任务的过程，任务拆解的目标是为了降低完成复杂任务的难度。例如以下就是一个复杂任务拆解过程，原始用户需求为Q：“请帮我查下西门吹雪给我发送了哪些邮件，并对这些邮件内容进行总结”，该需求可以拆分为两个最小子任务，分别是A:“1.查找我和西门吹雪之间通信的邮件列表”，“2.根据某份邮件列表，对这些邮件的主要内容进行总结”；

3.什么是最小子任务？最小子任务指的是在在邮箱项目中，在进行复杂任务拆解时拆解得到的最小任务单元。例如获取我和西门吹雪之间的通信列表、围绕某些邮件进行内容总结，就属于最小子任务。最小子任务不可再进行任务拆解，并且最小子任务往往都是较为简单的任务，是一些可以快速的实现的功能；

4.若一个任务拆解成了多个最小子任务，那么请注意这多个最小子任务的任务顺序，例如此前的例子中，查找邮件列表就应该发生在对邮件内容进行总结之前。

5.你是一名任务拆解助手，负责邮件项目中用户需求拆解。当一个用户需求可以拆解为多个最小子任务时，请对其进行最小子任务的拆解；而当一个用户需求本身就是一个最小子任务时，则无需对其进行拆解。


&emsp;&emsp;接下来继续创建Few-shot的提示示例。这里总共编写了四组难度和复杂度各不相同的四个需求进行拆解，具体提示示例如下：

In [4]:
example1_user = 'Q:请帮我查下西门吹雪给我发送了哪些邮件，并对这些邮件内容进行总结。'
example1_assistent = 'A:1.请帮我整理西门吹雪发给我的邮件列表。2.对某些给定的邮件进行邮件内容总结。'

example2_user = 'Q:请帮我查下最近一封未读邮件的发件人。'
example2_assistent = 'A:1.请帮我查下最近一封未读邮件的发件人。'

example3_user = 'Q:请分析我邮箱里全部已收到的邮件，告诉我最常联系的人。'
example3_assistent = 'A:1.请帮我整理邮箱里全部已收到邮件的邮件列表。2.根据某份邮件列表，分析并找出最常联系的人。'

example4_user = 'Q:请在我邮箱里查找6月5号到6号的未读邮件，并找出工作相关的未读邮件，请依次给这些未读邮件回复说我正在休假，同时请将这些邮件转发给我的助理西门吹雪。'
example4_assistent = 'A:1.请查找并整理我邮箱里6月5号到6号的未读邮件列表。2.根据某份邮件列表，查找并汇总其中和工作有关的列表。3.根据某份邮件列表，对其逐一进行回复，告诉他们我正在休假。4.根据某份邮件列表，将其逐一转发给我的助理西门吹雪。'

需要注意的是，这里我们统一通过Q&A的方式来提示模型不同内容的类型，同时拆解之后的每段文字也都以句号结尾，方便后续进行结构化字符串提取。在创建的四个提示实力中，1、3示例属于一般示例，对于大多数复杂需求来说，拆解为两个子任务基本就基本能够构成较好的自然语言编程引导，而第二个提示示例则是提醒模型，当需求较为简单时，不用进行需求拆解；而最后一个提示的示例，则是一个相对来说非常复杂的需求，总共拆解得到了4个子任务，是为了提醒模型在必要时，可以多拆分一些子任务。

In [5]:
req = '请查阅我邮箱里面未读邮件，从挑选有公司的邮件，并将其标签改为紧急'

In [6]:
system_message = [{"role": "system", "content": md_content_plus}]

In [7]:
decomp_few_shot = [{"role": "user", "content": example1_user},
                   {"role": "assistant", "content": example1_assistent}, 
                   {"role": "user", "content": example2_user}, 
                   {"role": "assistant", "content": example2_assistent}, 
                   {"role": "user", "content": example3_user}, 
                   {"role": "assistant", "content": example3_assistent}, 
                   {"role": "user", "content": example4_user}, 
                   {"role": "assistant", "content": example4_assistent}]

In [8]:
messages = system_message + decomp_few_shot
messages

[{'role': 'system',
  'content': '1.什么是邮箱项目？邮件项目本身由一系列的程序组成，核心功能是根据用户自然语言描述进行相应的邮箱操作，如查阅邮件、收发邮件等。该场景是我们自己构建的一个场景。\n主要实现的是对我们my_mail数据库里的数据进行增删查改。\n\n2.什么是任务拆解？在邮箱项目中，任务拆解指的是将用户需求拆解为若干个最小子任务的过程，任务拆解的目标是为了降低完成复杂任务的难度。例如以下就是一个复杂任务拆解过程，原始用户需求为Q：“请帮我查下西门吹雪给我发送了哪些邮件，并对这些邮件内容进行总结”，该需求可以拆分为两个最小子任务，分别是A:“1.查找我和西门吹雪之间通信的邮件列表”，“2.根据某份邮件列表，对这些邮件的主要内容进行总结”；\n\n3.什么是最小子任务？最小子任务指的是在在邮箱项目中，在进行复杂任务拆解时拆解得到的最小任务单元。例如获取我和端木天之间的通信列表、围绕某些邮件进行内容总结，就属于最小子任务。最小子任务不可再进行任务拆解，并且最小子任务往往都是较为简单的任务，是一些可以快速的实现的功能；\n\n4.若一个任务拆解成了多个最小子任务，那么请注意这多个最小子任务的任务顺序，例如此前的例子中，查找邮件列表就应该发生在对邮件内容进行总结之前。\n\n5.你是一名任务拆解助手，负责智能邮件项目中用户需求拆解。当一个用户需求可以拆解为多个最小子任务时，请对其进行最小子任务的拆解；而当一个用户需求本身就是一个最小子任务时，则无需对其进行拆解。'},
 {'role': 'user', 'content': 'Q:请帮我查下西门吹雪给我发送了哪些邮件，并对这些邮件内容进行总结。'},
 {'role': 'assistant', 'content': 'A:1.请帮我整理西门吹雪发给我的邮件列表。2.对某些给定的邮件进行邮件内容总结。'},
 {'role': 'user', 'content': 'Q:请帮我查下最近一封未读邮件的发件人。'},
 {'role': 'assistant', 'content': 'A:1.请帮我查下最近一封未读邮件的发件人。'},
 {'role': 'user', 'content': 'Q:请分析我邮箱里全部已收到的邮件，告诉我最常联系的人。'},
 {'role': '

In [224]:
with open('./functions/tested functions/%s.json' % 'decompose_messages', 'w') as f:
    json.dump(messages, f)

接下来输入用户需求：

In [9]:
messages.append({"role": "user", "content": req})

In [10]:
messages

[{'role': 'system',
  'content': '1.什么是邮箱项目？邮件项目本身由一系列的程序组成，核心功能是根据用户自然语言描述进行相应的邮箱操作，如查阅邮件、收发邮件等。该场景是我们自己构建的一个场景。\n主要实现的是对我们my_mail数据库里的数据进行增删查改。\n\n2.什么是任务拆解？在邮箱项目中，任务拆解指的是将用户需求拆解为若干个最小子任务的过程，任务拆解的目标是为了降低完成复杂任务的难度。例如以下就是一个复杂任务拆解过程，原始用户需求为Q：“请帮我查下西门吹雪给我发送了哪些邮件，并对这些邮件内容进行总结”，该需求可以拆分为两个最小子任务，分别是A:“1.查找我和西门吹雪之间通信的邮件列表”，“2.根据某份邮件列表，对这些邮件的主要内容进行总结”；\n\n3.什么是最小子任务？最小子任务指的是在在邮箱项目中，在进行复杂任务拆解时拆解得到的最小任务单元。例如获取我和端木天之间的通信列表、围绕某些邮件进行内容总结，就属于最小子任务。最小子任务不可再进行任务拆解，并且最小子任务往往都是较为简单的任务，是一些可以快速的实现的功能；\n\n4.若一个任务拆解成了多个最小子任务，那么请注意这多个最小子任务的任务顺序，例如此前的例子中，查找邮件列表就应该发生在对邮件内容进行总结之前。\n\n5.你是一名任务拆解助手，负责智能邮件项目中用户需求拆解。当一个用户需求可以拆解为多个最小子任务时，请对其进行最小子任务的拆解；而当一个用户需求本身就是一个最小子任务时，则无需对其进行拆解。'},
 {'role': 'user', 'content': 'Q:请帮我查下西门吹雪给我发送了哪些邮件，并对这些邮件内容进行总结。'},
 {'role': 'assistant', 'content': 'A:1.请帮我整理西门吹雪发给我的邮件列表。2.对某些给定的邮件进行邮件内容总结。'},
 {'role': 'user', 'content': 'Q:请帮我查下最近一封未读邮件的发件人。'},
 {'role': 'assistant', 'content': 'A:1.请帮我查下最近一封未读邮件的发件人。'},
 {'role': 'user', 'content': 'Q:请分析我邮箱里全部已收到的邮件，告诉我最常联系的人。'},
 {'role': '

In [12]:
response = client.chat.completions.create(
                model='glm-4',
                messages=messages
            )

In [15]:
print(response.choices[0].message.content)

对于这个用户需求，可以拆解为以下最小子任务：

1. 查阅邮箱中的未读邮件，并整理出未读邮件列表。
2. 从未读邮件列表中筛选出包含公司信息的邮件。
3. 对于筛选出的公司邮件，将其标签改为“紧急”。 

这些子任务按照顺序执行，可以满足用户的需求。


这里模型将原始需求拆分为了三个子任务。当然对于我们即将要执行的方案三来说，无论拆分多少个子任务，都是可以依次创建对应部分代码并进行合并的。

In [47]:
res=response.choices[0].message.content

In [17]:
# 使用正则表达式查找以1、2、3开始的句子
matches = re.findall(r'\d\.(.*?。)', res)

for match in matches:
    print(match)

 查阅邮箱中的未读邮件，并整理出未读邮件列表。
 从未读邮件列表中筛选出包含公司信息的邮件。
 对于筛选出的公司邮件，将其标签改为“紧急”。


In [18]:
sub_req1 = '查阅邮箱中的未读邮件，并整理出未读邮件列表。'
sub_req2 = '从未读邮件列表中筛选出包含公司信息的邮件。'
sub_req3 = '对于筛选出的公司邮件，将其标签改为“紧急”。'

至此，我们就完成了原始任务的拆解部分工作。当然，我们也可以将上述过程封装为一个完整的函数，方便之后进行灵活调用：

In [52]:
def get_decompose_results(req, model='glm-4'):
    """
    复杂需求拆解函数，能够将用户输入的复杂需求拆解为一系列更容易完成的子任务
    :param req: 必选参数，以字符串形式表示，用于表示用户输入的原始需求；
    :param model: 拆解需求所使用的大模型；
    :return：由子任务所组成的列表；
    """
    
    decompose_results = []
    
    with open('./functions/tested functions/decompose_messages.json', 'r') as f:
        decompose_messages = json.load(f)
        
    decompose_messages.append({"role": "user", "content": req})
    
    response = client.chat.completions.create(
                model=model,
                messages=decompose_messages
            )
    
    res = response.choices[0].message.content
    
    # 使用正则表达式查找以1、2、3开始的句子
    matches = re.findall(r'\d\.(.*?。)', res)

    for match in matches:
        decompose_results.append(match)
        
    return decompose_results

In [20]:
req

'请查阅我邮箱里面未读邮件，从挑选有公司的邮件，并将其标签改为紧急'

In [25]:
get_decompose_results(req)

[' 请帮我检索并整理出邮箱中的未读邮件列表。',
 ' 从未读邮件列表中筛选出包含公司信息的邮件。',
 ' 对筛选出的邮件进行标签添加，将其标记为“紧急”。']

In [31]:
def code_generate(req, few_shot='all', model='glm-4', g=globals(), detail=0):
    """
    Function calling外部函数自动创建函数，可以根据用户的需求，直接将其翻译为Chat模型可以直接调用的外部函数代码。
    :param req: 必要参数，字符串类型，表示输入的用户需求；
    :param few_shot: 可选参数，默认取值为字符串all，用于描述Few-shot提示示例的选取方案，当输入字符串all时，则代表提取当前外部函数库中全部测试过的函数作为Few-shot；\
    而如果输入的是一个包含了多个函数名称的list，则表示使用这些函数作为Few-shot。
    :param model: 可选参数，表示调用的Chat模型，默认选取glm-4；
    :param g: 可选参数，表示extract_function_code函数作用域，默认为globals()，即在当前操作空间全域内生效；
    :param detail: 可选参数，默认取值为0，还可以取值为1，表示extract_function_code函数打印新创建的外部函数细节；
    :return：新创建的函数名称。需要注意的是，在函数创建时，该函数也会在当前操作空间被定义，后续可以直接调用；
    """
    
    # 提取提示示例的函数名称
    if few_shot == 'all':
        few_shot_functions_name = show_functions(tested=True)
    elif type(few_shot) == list:
        few_shot_functions_name = few_shot
    # few_shot_functions = [globals()[name] for name in few_shot_functions_name]
    
    # 读取各阶段系统提示
    with open('./functions/tested functions/system_messages.json', 'r') as f:
        system_messages = json.load(f)
        
    # 各阶段提示message对象
    few_shot_messages_CM = []
    few_shot_messages_CD = []
    
    few_shot_messages = []
    
    # 先保存第一条消息，也就是system message
    few_shot_messages_CD += system_messages["system_message_CD"]
    few_shot_messages_CM += system_messages["system_message_CM"]
    few_shot_messages += system_messages["system_message"]

    # 创建不同阶段提示message
    for function_name in few_shot_functions_name:
        with open('./functions/tested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
            msg = json.load(f)
        few_shot_messages_CD += msg["stage1_CD"]
        few_shot_messages_CM += msg["stage1_CM"]
        few_shot_messages += msg['stage2']
        
    # 读取用户需求，作为第一阶段CD环节User content
    new_req_CD_input = req
    few_shot_messages_CD.append({"role": "user", "content": new_req_CD_input})
    
    print('第一阶段CD环节提示创建完毕，正在进行CD提示...')
    
    # 第一阶段CD环节Chat模型调用过程
    response = client.chat.completions.create(
                        model=model,
                        messages=few_shot_messages_CD)
    new_req_pi = response.choices[0].message.content
    
    #print("打印内容1："+new_req_pi)
    
    print('第一阶段CD环节提示完毕')
    
    # 第一阶段CM环节Messages创建
    new_req_CM_input = new_req_CD_input + new_req_pi
    few_shot_messages_CM.append({"role": "user", "content": new_req_CM_input})
    
    print('第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...')
    # 第一阶段CM环节Chat模型调用过程
    response = client.chat.completions.create(
                        model=model,
                        messages=few_shot_messages_CM)
    new_req_description = response.choices[0].message.content
    
    #print("打印内容2："+new_req_description)
    #return "test"

    
    print('第一阶段CM环节提示完毕')
    
    # 第二阶段Messages创建过程
    few_shot_messages.append({"role": "user", "content": new_req_description})
    
    print('第二阶段提示创建完毕，正在进行第二阶段提示...')
    
    # 第二阶段Chat模型调用过程
    response = client.chat.completions.create(
                        model=model,
                        messages=few_shot_messages)
    new_req_function = response.choices[0].message.content
    
    
    print('第二阶段提示完毕，准备运行函数并编写提示示例')
    
    # 提取函数并运行，创建函数名称对象，统一都写入untested文件夹内
    function_name = extract_function_code(s=new_req_function, detail=detail, g=g)
    
    print('新函数保存在./functions/untested functions/%s/%s_module.py文件中' % (function_name, function_name))
    
    # 创建该函数提示示例
    new_req_messages_CD = [
                          {"role": "user", "content": new_req_CD_input},
                          {"role": "assistant", "content": new_req_pi}
                         ]
    new_req_messages_CM = [
                          {"role": "user", "content": new_req_CM_input},
                          {"role": "assistant", "content":new_req_description}
                         ]
    
    with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), encoding='utf-8') as f:
        new_req_function = f.read()
    
    new_req_messages = [
                       {"role": "user", "content": new_req_description},
                       {"role": "assistant", "content":new_req_function}
                      ] 
    
    new_req_prompt = {
                     "stage1_CD": new_req_messages_CD,
                     "stage1_CM": new_req_messages_CM,
                     "stage2": new_req_messages
                    }   
    
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'w') as f:
        json.dump(new_req_prompt, f)
        
    print('新函数提示示例保存在./functions/untested functions/%s/%s_prompt.json文件中' % (function_name, function_name))
    print('done')
    return function_name

In [34]:
func1_name = code_generate(sub_req1, few_shot='all', g=globals(), detail=1)

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
An error occurred while executing the code:
name 'pymysql' is not defined
def fetch_unread_emails(user_id='me', max_emails=None):
    """
    获取指定用户ID的未读邮件列表。

    参数:
    - user_id: 字符串，表示要查询的邮箱用户ID，默认为'me'。
    - max_emails: 整数或None，表示要返回的最大邮件数量，如果为None，则返回所有未读邮件。

    返回:
    - JSON格式的字符串，包含未读邮件列表的详细信息。

    示例返回数据结构：
    {
        "unread_emails": [
            {
                "from_userid": "发件人ID",
                "date": "邮件发送日期",
                "subject": "邮件主题",
                "snippet": "邮件内容摘要",
                "status": "邮件状态",
                "label": "邮件标签"
            },
            ...
        ]
    }
    """

    # 数据库连接配置
    config = {
        'host': 'localhost',
        'user': 'gpt',
        'passwd': 'gpt',
        'db': 'my_mail',
        'charset': 'utf8'
    }

    # 建立数据库连接
    connection = pymysql.connect(**config)
    try:


In [36]:
func2_name = code_generate(sub_req2, few_shot='all', g=globals(), detail=1)

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
An error occurred while executing the code:
local variable 'connection' referenced before assignment
def retrieve_recent_emails(n, user_id='me'):
    """
    查询指定用户邮箱中最近的几封邮件信息。

    参数:
    n: 要查询的邮件数量，数据格式为字符串类型。
    user_id: 要查询的用户的ID，默认值为'me'，表示当前授权的用户。

    返回:
    一个JSON格式的对象，包含查询到的邮件列表。
    每封邮件包含以下信息：'from_userid', 'date', 'subject', 'snippet'。

    示例返回:
    {
        "emails": [
            {"from_userid": "sender1", "date": "2023-04-01", "subject": "Subject 1", "snippet": "Mail content snippet 1"},
            {"from_userid": "sender2", "date": "2023-04-02", "subject": "Subject 2", "snippet": "Mail content snippet 2"}
        ]
    }
    """
    try:
        # 建立数据库连接
        connection = pymysql.connect(**DB_CONFIG)
        with connection.cursor() as cursor:
            # 查询最近的n封邮件
            sql_query = f"SELECT from_userid, date, subject, s

In [37]:
func3_name = code_generate(sub_req3, few_shot='all', g=globals(), detail=1)

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
def update_emails_tag(n, user_id='me', tag='紧急'):
    """
    更新邮箱中最近的n封邮件的标签为指定的tag。

    参数:
    n: 要更新的邮件数量，数据格式为整型。
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。
    tag: 要设置的邮件标签。默认值是'紧急'，数据格式为字符串类型。

    返回:
    一个JSON格式对象，包含更新后的邮件信息。

    注意:
    - 这个函数假设你已经有了数据库连接的相关配置，并且数据库中有一个名为`mailbox`的表。
    - 请确保在真实环境中处理SQL注入的安全性问题。
    - 这个函数没有实现事务回滚，如果更新失败，需要手动处理异常。
    """

    # 假设这是你的数据库连接配置
    config = {
        'host': 'localhost',
        'user': 'gpt',
        'passwd': 'gpt',
        'db': 'my_mail',
        'charset': 'utf8'
    }

    try:
        # 建立数据库连接
        connection = pymysql.connect(**config)
        with connection.cursor() as cursor:
            # 查询最近的n封邮件
            sql_query = f"SELECT * FROM mailbox WHERE userid = '{user_id}' ORDER BY date DESC LIMIT {n}"
            cursor.execute(sql_query)
            emails = cursor.fetchall

In [38]:
with open('./functions/untested functions/%s/%s_module.py' % (func1_name, func1_name), encoding='utf-8') as f:
    func1_str = f.read()
    
with open('./functions/untested functions/%s/%s_module.py' % (func2_name, func2_name), encoding='utf-8') as f:
    func2_str = f.read()
    
with open('./functions/untested functions/%s/%s_module.py' % (func3_name, func3_name), encoding='utf-8') as f:
    func3_str = f.read()

然后输入到大模型中，令其读取这三段代码并对其进行合并：

In [39]:
response = client.chat.completions.create(
  model='glm-4',
  messages=[
      {"role": "system", "content": '以下是函数1的代码：%s' % func1_str},
      {"role": "system", "content": '以下是函数2的代码：%s' % func2_str},
      {"role": "system", "content": '以下是函数3的代码：%s' % func3_str},
      {"role": "user", "content": '请将函数1、函数2和函数3的代码功能整合到一个函数中，要求该函数功能完整，并且可以满足以下需求：%s' % req}
  ]
)

In [40]:
python_content = response.choices[0].message.content
python_content

'下面是一个整合后的函数，它首先获取未读邮件列表，然后挑选出发件人包含“公司”关键词的邮件，并将这些邮件的标签更新为“紧急”。请注意，为了防止SQL注入，我们在使用字符串格式化时使用了参数化查询。\n\n```python\nimport json\nimport pymysql\n\n# 假设这是你的数据库连接配置\nDB_CONFIG = {\n    \'host\': \'localhost\',\n    \'user\': \'gpt\',\n    \'passwd\': \'gpt\',\n    \'db\': \'my_mail\',\n    \'charset\': \'utf8\'\n}\n\ndef manage_emails(user_id=\'me\', max_emails=None, tag=\'紧急\', company_keyword=\'公司\'):\n    """\n    获取并处理指定用户ID的未读邮件列表，挑选含有公司关键词的邮件，并将其标签更新为紧急。\n\n    参数:\n    - user_id: 字符串，表示要查询的邮箱用户ID，默认为\'me\'。\n    - max_emails: 整数或None，表示要返回的最大邮件数量，如果为None，则返回所有未读邮件。\n    - tag: 字符串，表示要更新的邮件标签，默认为\'紧急\'。\n    - company_keyword: 字符串，表示筛选邮件时使用的公司关键词，默认为\'公司\'。\n\n    返回:\n    - JSON格式的字符串，包含处理后的邮件列表和可能出现的错误信息。\n    """\n\n    try:\n        # 建立数据库连接\n        connection = pymysql.connect(**DB_CONFIG)\n        with connection.cursor() as cursor:\n            # 获取未读邮件列表\n            query = "SELECT * FROM mailbox WHERE userid = %s AND status = \'未读\'"\n            if max_emails is not None:

In [41]:
extract_function_code(python_content, detail=1, tested=False, g=globals())

An error occurred while executing the code:
local variable 'connection' referenced before assignment
def manage_emails(user_id='me', max_emails=None, tag='紧急', company_keyword='公司'):
    """
    获取并处理指定用户ID的未读邮件列表，挑选含有公司关键词的邮件，并将其标签更新为紧急。

    参数:
    - user_id: 字符串，表示要查询的邮箱用户ID，默认为'me'。
    - max_emails: 整数或None，表示要返回的最大邮件数量，如果为None，则返回所有未读邮件。
    - tag: 字符串，表示要更新的邮件标签，默认为'紧急'。
    - company_keyword: 字符串，表示筛选邮件时使用的公司关键词，默认为'公司'。

    返回:
    - JSON格式的字符串，包含处理后的邮件列表和可能出现的错误信息。
    """

    try:
        # 建立数据库连接
        connection = pymysql.connect(**DB_CONFIG)
        with connection.cursor() as cursor:
            # 获取未读邮件列表
            query = "SELECT * FROM mailbox WHERE userid = %s AND status = '未读'"
            if max_emails is not None:
                query += " LIMIT %s"
                cursor.execute(query, (user_id, max_emails))
            else:
                cursor.execute(query, (user_id,))

            # 获取查询结果
            rows = cursor.fetchall()

            # 筛选含有公司

'manage_emails'

In [ ]:
print(manage_emails('me', 10, '紧急', '公司'))

&emsp;&emsp;需要注意的是，实际操作过程中，这种合并方式的准确率其实和子函数的代码质量有很大的关系，需要注意的是，此时由于大模型由于缺少了LtM提示流程，因此对原始编程需求理解的并不深，只能根据子函数的代码来推断原始需求的实现过程，因此一旦子函数本身质量不高，最终合并得到的函数的准确率也会比较低。而同时我们也知道，子函数的创建本身就是没有审核的，因此子函数的代码质量也极有可能是层次不齐的，因此总的来看，这种合并方式仍然存在一些不太可控的风险。

### 4.4 复杂任务的自动编程落地

&emsp;&emsp;我们需要修改此前定义的code_generate函数，为其增加一个system_messages参数。该参数默认取值为None，表示导入本地保存的system_messages.json对象，否则输入外部messages作为system_messages:

In [32]:
def code_generate(req, few_shot='all', model='glm-4', g=globals(), detail=0, system_messages=None):
    """
    Function calling外部函数自动创建函数，可以根据用户的需求，直接将其翻译为Chat模型可以直接调用的外部函数代码。
    :param req: 必要参数，字符串类型，表示输入的用户需求；
    :param few_shot: 可选参数，默认取值为字符串all，用于描述Few-shot提示示例的选取方案，当输入字符串all时，则代表提取当前外部函数库中全部测试过的函数作为Few-shot；\
    而如果输入的是一个包含了多个函数名称的list，则表示使用这些函数作为Few-shot。
    :param model: 可选参数，表示调用的Chat模型，默认选取glm-4；
    :param g: 可选参数，表示extract_function_code函数作用域，默认为globals()，即在当前操作空间全域内生效；
    :param detail: 可选参数，默认取值为0，还可以取值为1，表示extract_function_code函数打印新创建的外部函数细节；
    :param system_messages: 可选参数，默认取值为None，此时读取本地system_messages.json作为系统信息，也可以自定义；
    :return：新创建的函数名称。需要注意的是，在函数创建时，该函数也会在当前操作空间被定义，后续可以直接调用；
    """
    
    # 提取提示示例的函数名称
    if few_shot == 'all':
        few_shot_functions_name = show_functions(tested=True)
    elif type(few_shot) == list:
        few_shot_functions_name = few_shot
    # few_shot_functions = [globals()[name] for name in few_shot_functions_name]
    
    # 读取各阶段系统提示
    if system_messages == None:
        with open('./functions/tested functions/system_messages.json', 'r') as f:
            system_messages = json.load(f)
        
    # 各阶段提示message对象
    few_shot_messages_CM = []
    few_shot_messages_CD = []
    few_shot_messages = []
    
    # 先保存第一条消息，也就是system message
    few_shot_messages_CD += system_messages["system_message_CD"]
    few_shot_messages_CM += system_messages["system_message_CM"]
    few_shot_messages += system_messages["system_message"]

    # 创建不同阶段提示message
    for function_name in few_shot_functions_name:
        with open('./functions/tested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
            msg = json.load(f)
        few_shot_messages_CD += msg["stage1_CD"]
        few_shot_messages_CM += msg["stage1_CM"]
        few_shot_messages += msg['stage2']
        
    # 读取用户需求，作为第一阶段CD环节User content
    new_req_CD_input = req
    few_shot_messages_CD.append({"role": "user", "content": new_req_CD_input})
    
    print('第一阶段CD环节提示创建完毕，正在进行CD提示...')
    
    # 第一阶段CD环节Chat模型调用过程
    response = client.chat.completions.create(
                  model=model,
                  messages=few_shot_messages_CD
                )
    new_req_pi = response.choices[0].message.content
    
    print('第一阶段CD环节提示完毕')
    
    # 第一阶段CM环节Messages创建
    new_req_CM_input = new_req_CD_input + new_req_pi
    few_shot_messages_CM.append({"role": "user", "content": new_req_CM_input})
    
    print('第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...')
    # 第一阶段CM环节Chat模型调用过程
    response = client.chat.completions.create(
                      model=model,
                      messages=few_shot_messages_CM
                    )
    new_req_description = response.choices[0].message.content
    
    print('第一阶段CM环节提示完毕')
    
    # 第二阶段Messages创建过程
    few_shot_messages.append({"role": "user", "content": new_req_description})
    
    print('第二阶段提示创建完毕，正在进行第二阶段提示...')
    
    # 第二阶段Chat模型调用过程
    response = client.chat.completions.create(
                  model=model,
                  messages=few_shot_messages
                )
    new_req_function = response.choices[0].message.content
    
    print('第二阶段提示完毕，准备运行函数并编写提示示例')
    
    # 提取函数并运行，创建函数名称对象，统一都写入untested文件夹内
    function_name = extract_function_code(s=new_req_function, detail=detail, g=g)
    
    print('新函数保存在./functions/untested functions/%s/%s_module.py文件中' % (function_name, function_name))
    
    # 创建该函数提示示例
    new_req_messages_CD = [
                          {"role": "user", "content": new_req_CD_input},
                          {"role": "assistant", "content": new_req_pi}
                         ]
    new_req_messages_CM = [
                          {"role": "user", "content": new_req_CM_input},
                          {"role": "assistant", "content":new_req_description}
                         ]
    
    with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), encoding='utf-8') as f:
        new_req_function = f.read()
    
    new_req_messages = [
                       {"role": "user", "content": new_req_description},
                       {"role": "assistant", "content":new_req_function}
                      ] 
    
    new_req_prompt = {
                     "stage1_CD": new_req_messages_CD,
                     "stage1_CM": new_req_messages_CM,
                     "stage2": new_req_messages
                    }   
    
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'w') as f:
        json.dump(new_req_prompt, f)
        
    print('新函数提示示例保存在./functions/untested functions/%s/%s_prompt.json文件中' % (function_name, function_name))
    print('done')
    return function_name

In [67]:
with open('./functions/tested functions/system_messages.json', 'r') as f:
    system_messages = json.load(f)

In [68]:
res="""
对于这个用户需求，可以拆解为以下最小子任务：

1. 查阅邮箱中的未读邮件，并整理出未读邮件列表。
2. 从未读邮件列表中筛选出包含公司信息的邮件。
3. 对于筛选出的公司邮件，将其标签改为“紧急”。 
"""

In [69]:
decompose_description = ('对于当前编程需求，可以拆解为若干个子需求，也就是：%s。这些子需求的实现方式可以参考：' % res ) + func1_str + func2_str + func3_str

In [70]:
decompose_description

'对于当前编程需求，可以拆解为若干个子需求，也就是：\n对于这个用户需求，可以拆解为以下最小子任务：\n\n1. 查阅邮箱中的未读邮件，并整理出未读邮件列表。\n2. 从未读邮件列表中筛选出包含公司信息的邮件。\n3. 对于筛选出的公司邮件，将其标签改为“紧急”。 \n。这些子需求的实现方式可以参考：def fetch_unread_emails(user_id=\'me\', max_emails=None):\n    """\n    获取指定用户ID的未读邮件列表。\n\n    参数:\n    - user_id: 字符串，表示要查询的邮箱用户ID，默认为\'me\'。\n    - max_emails: 整数或None，表示要返回的最大邮件数量，如果为None，则返回所有未读邮件。\n\n    返回:\n    - JSON格式的字符串，包含未读邮件列表的详细信息。\n\n    示例返回数据结构：\n    {\n        "unread_emails": [\n            {\n                "from_userid": "发件人ID",\n                "date": "邮件发送日期",\n                "subject": "邮件主题",\n                "snippet": "邮件内容摘要",\n                "status": "邮件状态",\n                "label": "邮件标签"\n            },\n            ...\n        ]\n    }\n    """\n\n    # 数据库连接配置\n    config = {\n        \'host\': \'localhost\',\n        \'user\': \'gpt\',\n        \'passwd\': \'gpt\',\n        \'db\': \'my_mail\',\n        \'charset\': \'utf8\'\n    }\n\n    # 建立数据库连接\n    connection = pymysql.connect(**config)\n  

In [71]:
system_messages['system_message'].append({'role': 'system', 'content': decompose_description})

In [ ]:
然后以此作为system_message带入到code_generate中进行代码创建：

In [72]:
code_generate(req, few_shot='all', model='glm-4', g=globals(), detail=0, system_messages=system_messages)

第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
An error occurred while executing the code:
local variable 'cursor' referenced before assignment
The function name is:update_mail_label
新函数保存在./functions/untested functions/update_mail_label/update_mail_label_module.py文件中
新函数提示示例保存在./functions/untested functions/update_mail_label/update_mail_label_prompt.json文件中
done


'update_mail_label'

In [73]:
update_mail_label?

Signature: update_mail_label(user_id='me', keyword='公司', new_label='紧急')
Docstring:
查询指定用户邮箱中包含特定关键词的未读邮件，并将其标签更改为指定的值。

参数：
user_id (str): 要查询的邮箱用户ID，默认值为'me'。
keyword (str): 要查找的邮件内容中的关键词，默认值为“公司”。
new_label (str): 要修改的邮件标签，默认值为“紧急”。

返回：
json对象: 包含修改后的邮件信息的json格式对象。
File:      Dynamically generated function. No source code available.
Type:      function

**生成函数没问题我这儿就不执行了，因为影响数据库中的数据**

In [ ]:
messages = [{"role": "system", "content": "我的邮箱号是me"},
            {"role": "user", "content": req}]

In [ ]:
run_conversation(messages, functions_list=[label_meeting_emails], model="glm-4")

&emsp;&emsp;这里需要注意的是，我们之所以考虑采用code_generate函数进行对应代码创建，核心原因在于该函数会创建一份完整的prompt提示示例，便于搭配function_test对其进行审查。接下来，我们就尝试围绕修改function_test的测试流程，是的其能够对拆分多个子需求之后的函数创建过程进行审查。

&emsp;&emsp;首先需要修改的就是prompt_modified函数，为了更好的协调function_test和code_generate共同进行代码创建和审查，我们需要将这些函数的相关参数进行统一，这里对于prompt_modified函数的修改主要是添加一个system_messages参数，用于表示此时是围绕简单需求进行提示审查还是围绕拆解需求并进行分段代码创建的流程进行提示审查，和code_generate函数类似，默认情况都是取值为None，表示围绕简单需求进行提示审查。而当system_messages取值不为None时，则会提供一个名为《复杂需求推理链》的文档作为提示审查函数外部挂载文档，相比《推理链》文档，该文档只修改了一个部分，即将原先的作为正确示例的prompt.json文档修改为此前我们成功创建的label_meeting_emails函数对应的提示示例，该函数的提示示例（配合复杂需求拆解的system_message）能够更好的表示复杂需求拆解和代码合并流程：

**我这儿单独提供了一个高质量的prompt：update_mail_lable_prompt.json**

In [79]:
 with open('./functions/untested functions/update_mail_lable_plus/update_mail_lable_plus_prompt.json' , 'r') as f:
    data = json.load(f)

In [80]:
data

{'stage1_CD': [{'role': 'user', 'content': '请查阅我邮箱里面未读邮件，从挑选有公司的邮件，并将其标签改为紧急'},
  {'role': 'assistant',
   'content': '请查阅我邮箱里面未读邮件，从挑选有公司的邮件，并将其标签改为紧急当前需求中可以作为函数参数的是：1. 查询谁的邮箱；2. 查找带有特定关键词的邮件（如“公司”）；3. 要修改的邮件标签（如“紧急”）'}],
 'stage1_CM': [{'role': 'user',
   'content': '请查阅我邮箱里面未读邮件，从挑选有公司的邮件，并将其标签改为紧急请查阅我邮箱里面未读邮件，从挑选有公司的邮件，并将其标签改为紧急当前需求中可以作为函数参数的是：1. 查询谁的邮箱；2. 查找带有特定关键词的邮件（如“公司”）；3. 要修改的邮件标签（如“紧急”）'},
  {'role': 'assistant',
   'content': "请帮我编写一个python函数，查阅我邮箱里面未读邮件，从挑选有公司的邮件，并将其标签改为紧急，函数要求如下：                  1.函数参数user_id,keyword,new_label   ，三个参数都是字符串类型，其中keyword表示关键词，tnew_label表示要修改为的新标签，user_id表示要检索邮箱的用户的ID。默认值是'me'，表示当前授权的用户；                  2.函数返回结果是添加标签后的邮件列表，返回结果本身必须是一个json格式对象；                  3.请将全部功能封装在一个函数内；                  4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"}],
 'stage2': [{'role': 'user',
   'content': "请帮我编写一个python函数，查阅我邮箱里面未读邮件，从挑选有公司的邮件，并将其标签改为紧急，函数要求如下：                  1.函数参数user_id,keyword,new_label   ，三个参数都是字符串类型，其中keyword表示关键词，tnew

我们用这个数据去修改《复杂任务推理链》

重新优化prompt_modified方法

In [33]:
def prompt_modified(function_name, system_content='复杂任务推理链.md', model="glm-4", g=globals(), system_messages=None):
    """
    邮箱项目的外部函数审查函数，用于审查外部函数创建流程提示是否正确以及最终创建的代码是否正确
    :param function_name: 必要参数，字符串类型，表示审查对象名称；
    :param system_content: 可选参数，默认取值为字符串推理链修改.md，表示此时审查函数外部挂载文档名称，需要是markdwon格式文档；
    :param model: 可选参数，表示调用的Chat模型，默认选取gpt-4；
    :param g: 可选参数，表示extract_function_code函数作用域，默认为globals()，即在当前操作空间全域内生效；
    :param system_messages: 可选参数，默认取值为None，此时读取本地system_messages.json作为系统信息，也可以自定义；
    :return：审查结束后新创建的函数名称
    """
    print("正在执行审查函数，审查对象：%s" % function_name)
    
    if system_messages != None:
        system_content='复杂任务推理链.md'
        
    with open(system_content, 'r', encoding='utf-8') as f:
        md_content = f.read()  
        
    # 读取原函数全部提示内容
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'r') as f:
        msg = json.load(f)
    
    # 将其保存为字符串
    msg_str = json.dumps(msg)
    
    # 进行审查
    response = client.chat.completions.create(
                    model=model,
                    messages=[
                    {"role": "system", "content": md_content},
                    {"role": "user", "content": '以下是一个错误的邮件的推理链，请你按照要求对其进行修改：%s' % msg_str}
                    ]
                )
    
    modified_result = response.choices[0].message.content
    
    def extract_json(s):
        pattern = r'```[jJ][sS][oO][nN]\s*({.*?})\s*```'
        match = re.search(pattern, s, re.DOTALL)
        if match:
            return match.group(1)
        else:
            return s
    
    modified_json = extract_json(modified_result)
    
    # 提取函数源码
    code = json.loads(modified_json)['stage2'][1]['content']
    
    # 提取函数名
    match = re.search(r'def (\w+)', code)
    function_name = match.group(1)
    
    print("审查结束，新的函数名称为：%s。\n正在运行该函数定义过程，并保存函数源码与prompt" % function_name)
    
    exec(code, g)
    
    # 在untested文件夹内创建函数同名文件夹
    directory = './functions/untested functions/%s' % function_name
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    # 写入函数
    with open('./functions/untested functions/%s/%s_module.py' % (function_name, function_name), 'w', encoding='utf-8') as f:
        f.write(code)
        
    # 写入提示
    with open('./functions/untested functions/%s/%s_prompt.json' % (function_name, function_name), 'w') as f:
        json.dump(json.loads(modified_json), f)
    
    print('新函数提示示例保存在./functions/untested functions/%s/%s_prompt.json文件中' % (function_name, function_name))
    print("%s函数已在当前操作空间定义，可以进行效果测试" % function_name)
    
    return function_name

&emsp;&emsp;据此，我们就可以进一步围绕function_test函数进行修改。该函数的修改过程并不复杂，核心是需要将审查过程所涉及的一系列的函数（包括自己）添加一个system_messages的参数，同样，默认情况下取值为None，其他时候该参数的取值将输入到code_genetate和prompt_modified函数中：

In [81]:
def function_test(function_name, req, few_shot, model="glm-4", g=globals(), system_messages=None):

    def test_messages(ueser_content):
        messages = [{"role": "system", "content": "我的邮箱号是me"},
                    {"role": "user", "content": ueser_content}]
        return messages
            
    messages = test_messages(req)
    
    new_function = globals()[function_name]
    functions_list = [new_function]
    
    print("根据既定用户需求req进行%s函数功能测试，请确保当该函数已经在当前操作空间定义..." % function_name)
    
    # 有可能在run_conversation环节报错
    # 若没报错，则运行：
    try:
        final_response = run_conversation(messages=messages, functions_list=functions_list, model=model)
        print("当前函数运行结果：'%s'" % final_response)
        
        feedback = input("函数功能是否满足要求 (yes/no)? ")
        if feedback.lower() == 'yes':
            print("函数功能通过测试，正在将函数写入tested文件夹")
            remove_to_tested(function_name)
            print('done')
        else:
            next_step = input("函数功能未通过测试，是1.需要再次进行测试，还是2.进入debug流程？")
            if next_step == '1':
                print("准备再次测试...")
                function_test(function_name=function_name, req=req, few_shot=few_shot, g=g, system_messages=system_messages)
            else:
                solution = input("请选择debug方案：\n1.再次执行函数创建流程，并测试结果；\n2.执行审查函数\
                \n3.重新输入用户需求；\n4.退出程序，进行手动尝试")
                if solution == '1':
                    # 再次运行函数创建过程
                    print("好的，正在尝试再次创建函数，请稍等...")
                    few_shot_str = input("准备再次测试，请问是1.采用此前Few-shot方案，还是2.带入全部函数示例进行Few-shot？")
                    if few_shot_str == '1':
                        req_auto_func(req=req, few_shot=few_shot, model=model, g=g)
                    else:
                        req_auto_func(req=req, few_shot='all', model=model, g=g)
                elif solution == '2':
                    # 执行审查函数
                    print("好的，执行审查函数，请稍等...")
                    function_name = prompt_modified(function_name=function_name, model="gpt-4", g=g, system_messages=system_messages)
                    # 接下来带入进行测试
                    print("新函数已创建，接下来带入进行测试...")
                    function_test(function_name=function_name, req=req, few_shot=few_shot, g=g, system_messages=system_messages)
                elif solution == '3':
                    new_req = input("好的，请再次输入用户需求，请注意，用户需求描述方法将极大程度影响最终函数创建结果。")
                    few_shot_str = input("接下来如何运行代码创建函数？1.采用此前Few-shot方案；\n2.使用全部外部函数作为Few-shot")
                    if few_shot_str == '1':
                        req_auto_func(req=req, few_shot=few_shot, model=model, g=g)
                    else:
                        req_auto_func(req=req, few_shot='all', model=model, g=g)
                elif solution == '4':
                    print("好的，预祝debug顺利~")
        
    # run_conversation报错时则运行：
    except Exception as e:
        next_step = input("run_conversation无法正常运行，接下来是1.再次运行运行run_conversation，还是2.进入debug流程？")
        if next_step == '1':
            function_test(function_name=function_name, req=req, few_shot=few_shot, g=g, system_messages=system_messages)
        else:
            solution = input("请选择debug方案：\n1.再次执行函数创建流程，并测试结果；\n2.执行审查函数\
            \n3.重新输入用户需求；\n4.退出程序，进行手动尝试")
            if solution == '1':
                # 再次运行函数创建过程
                print("好的，正在尝试再次创建函数，请稍等...")
                few_shot_str = input("准备再次测试，请问是1.采用此前Few-shot方案，还是2.带入全部函数示例进行Few-shot？")
                if few_shot_str == '1':
                    req_auto_func(req=req, few_shot=few_shot, model=model, g=g)
                else:
                    req_auto_func(req=req, few_shot='all', model=model, g=g)
            elif solution == '2':
                # 执行审查函数
                print("好的，执行审查函数，请稍等...")
                max_attempts = 3
                attempts = 0

                while attempts < max_attempts:
                    try:
                        function_name = prompt_modified(function_name=function_name, model="gpt-3.5-turbo-16k-0613", g=g, system_messages=system_messages)
                        break  # 如果代码成功执行，跳出循环
                    except Exception as e:
                        attempts += 1  # 增加尝试次数
                        print("发生错误：", e)
                        if attempts == max_attempts:
                            print("已达到最大尝试次数，程序终止。")
                            raise  # 重新引发最后一个异常
                        else:
                            print("正在重新运行审查程序...")
                # 接下来带入进行测试
                print("新函数已创建，接下来带入进行测试...")
                function_test(function_name=function_name, req=req, few_shot=few_shot, g=g, system_messages=system_messages)
            elif solution == '3':
                new_req = input("好的，请再次输入用户需求，请注意，用户需求描述方法将极大程度影响最终函数创建结果。")
                few_shot_str = input("接下来如何运行代码创建函数？1.采用此前Few-shot方案；\n2.使用全部外部函数作为Few-shot")
                if few_shot_str == '1':
                    req_auto_func(req=req, few_shot=few_shot, model=model, g=g)
                else:
                    req_auto_func(req=req, few_shot='all', model=model, g=g)
            elif solution == '4':
                print("好的，预祝debug顺利~")

&emsp;&emsp;至此，我们就完成了code_generate函数和function_test函数修改。当然，为了让整个自动编程流程能够更加自动，接下来我们进一步对req_auto_func函数进行修改。此时该函数不仅承担同时运行code_generate函数和function_test函数的功能，更承担着对原始任务进行拆解的功能，具体函数修改内容如下：

In [77]:
import time
def req_auto_func(req, few_shot='all', model='glm-4', g=globals(), detail=0):
    # 默认情况下system_messages = None
    system_messages = None
    
    # 尝试进行任务拆解
    try:
        decompose_results = get_decompose_results(req=req, model=model)
        print("代码能执行到这儿吗？")
    except Exception as e:
        print(e)
        print('暂停1分钟后继续调用模型')
        #time.sleep(60)
        decompose_results = get_decompose_results(req=req, model=model)
        
    # 如果只拆解得到多个任务，则创建新的基于任务拆解得到的system_message
    if len(decompose_results) != 1:
        print('原始需求将拆分为多个子需求并进行分段代码创建与合并')
        # 读取原始system_message
        with open('./functions/tested functions/system_messages.json', 'r') as f:
            system_messages = json.load(f)
            
        # 用于存储全部需求的函数代码
        sub_func_all = ''
        # 用于存储全部需求
        sub_req_all = ''
        # 计数器
        i = 0
        
        for sub_req in decompose_results:
            i += 1
            # 每个需求依次创建子函数
            print('第%s个子需求为：%s' % (i, sub_req))
            print('正在创建对应子函数')
            try:
                sub_func_name = code_generate(sub_req, few_shot=few_shot, g=g, detail=detail, model=model)
            except Exception as e:
                print(e)
                print('暂停1分钟后继续调用模型')
                time.sleep(60)
                sub_func_name = code_generate(sub_req, few_shot=few_shot, g=g, detail=detail, model=model)
                
            # 读取子函数源码
            with open('./functions/untested functions/%s/%s_module.py' % (sub_func_name, sub_func_name), encoding='utf-8') as f:
                sub_func_str = f.read()
            # 对子函数源码进行拼接
            sub_func_all += sub_func_str
            
            # 按顺序拼接子需求
            sub_req_all += str(i)+('.')
            sub_req_all += sub_req            
        
        print('子需求对应的子函数全部创建完毕，接下来进入到原始需求函数创建过程...')
        # 添加一个system_message
        decompose_description = '对于当前编程需求，可以拆解为若干个子需求，也就是：%s。这些子需求的实现方式可以参考如下代码：%s' % (sub_req_all, sub_func_all)
        system_messages['system_message'].append({'role': 'system', 'content': decompose_description})
    
    # 进行代码创建和代码审查
    try:
        function_name = code_generate(req=req, few_shot=few_shot, model=model, g=g, detail=detail, system_messages=system_messages)
    except Exception as e:
        print(e)
        print('暂停1分钟后继续调用模型')
        time.sleep(60)
        function_name = code_generate(req=req, few_shot=few_shot, model=model, g=g, detail=detail, system_messages=system_messages)
        
    function_test(function_name=function_name, req=req, few_shot=few_shot, model=model, g=g, system_messages=system_messages)

In [ ]:
# GLM-4模型，GPT-4

In [88]:
req = '在我的邮箱里，查一下陆小凤给我发的邮件，并把标签改为“季度总结”'

In [89]:
req_auto_func(req=req)

代码能执行到这儿吗？
原始需求将拆分为多个子需求并进行分段代码创建与合并
第1个子需求为：请帮我查找陆小凤发给我的邮件列表。
正在创建对应子函数
第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
The function name is:get_latest_emails
新函数保存在./functions/untested functions/get_latest_emails/get_latest_emails_module.py文件中
新函数提示示例保存在./functions/untested functions/get_latest_emails/get_latest_emails_prompt.json文件中
done
第2个子需求为：根据邮件列表，将这些邮件的标签修改为“季度总结”。
正在创建对应子函数
第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
'NoneType' object has no attribute 'group'
暂停1分钟后继续调用模型
第一阶段CD环节提示创建完毕，正在进行CD提示...
第一阶段CD环节提示完毕
第一阶段CM环节提示创建完毕，正在进行第一阶段CM提示...
第一阶段CM环节提示完毕
第二阶段提示创建完毕，正在进行第二阶段提示...
第二阶段提示完毕，准备运行函数并编写提示示例
An error occurred while executing the code:
local variable 'connection' referenced before assignment
The function name is:update_email_labels
新函数保存在./functions/untested functions/update_email_labels/update_ema

run_conversation无法正常运行，接下来是1.再次运行运行run_conversation，还是2.进入debug流程？ 2
请选择debug方案：
1.再次执行函数创建流程，并测试结果；
2.执行审查函数            
3.重新输入用户需求；
4.退出程序，进行手动尝试 4


好的，预祝debug顺利~
